<a id='top'></a>

# Player Golden ID of Football Datasets
##### Notebook to join the scraped and engineered player datasets inclduing the performance dataset from [FBref](https://fbref.com/en/) (provided by [StatsBomb](https://statsbomb.com/)), [TransferMarkt](https://www.transfermarkt.co.uk/) bio, status, historical player values, and [recorded transfer](https://github.com/ewenme/transfers) datasets, and player salaries dataset from [Capology](https://www.capology.com/), through the [record-linkage](https://pypi.org/project/recordlinkage/) library, to create one, unified source of information, that can be used for for further analysis of players performance statistics and financial valuations, to be used as part of an Expected Transfer (xTransfer) model.

### By [Edd Webster](https://www.twitter.com/eddwebster)
Notebook first written: 03/08/2021<br>
Notebook last updated: 04/09/2021

![title](../../img/logos/fbref-logo-banner.png)

![title](../../img/logos/stats-bomb-logo.png)

![title](../../img/logos/transfermarkt-logo-banner.png)

![title](../../img/logos/capology-logo.jpeg)

![title](../../img/logos/elo.png)

Click [here](#section5) to jump straight to the Exploratory Data Analysis section and skip the [Task Brief](#section2), [Data Sources](#section3), and [Data Engineering](#section4) sections. Or click [here](#section6) to jump straight to the Conclusion.

___

<a id='sectionintro'></a>

## <a id='import_libraries'>Introduction</a>
This notebook joins datasets scraped from [FBref](https://fbref.com/en/) provided by [StatsBomb](https://statsbomb.com/), [TransferMarkt](https://www.transfermarkt.co.uk/) estimated player values and [recorded transfer](https://github.com/ewenme/transfers) datasets, and player salaries dataset from [Capology](https://www.capology.com/), through the [record-linkage](https://pypi.org/project/recordlinkage/) library, to create one, unified source of information, that can be used for for further analysis of players performance statistics and financial valuations, to be used as part of an Expected Transfer (xTransfer) model. This landed data is then manipulated as DataFrames using [pandas](http://pandas.pydata.org/).

For more information about this notebook and the author, I'm available through all the following channels:
*    [eddwebster.com](https://www.eddwebster.com/);
*    edd.j.webster@gmail.com;
*    [@eddwebster](https://www.twitter.com/eddwebster);
*    [linkedin.com/in/eddwebster](https://www.linkedin.com/in/eddwebster/);
*    [github/eddwebster](https://github.com/eddwebster/);
*    [public.tableau.com/profile/edd.webster](https://public.tableau.com/profile/edd.webster);
*    [kaggle.com/eddwebster](https://www.kaggle.com/eddwebster); and
*    [hackerrank.com/eddwebster](https://www.hackerrank.com/eddwebster).

![title](../../img/fifa21eddwebsterbanner.png)

The accompanying GitHub repository for this notebook can be found [here](https://github.com/eddwebster/football_analytics) and a static version of this notebook can be found [here](https://nbviewer.jupyter.org/github/eddwebster/football_analytics/blob/master/notebooks/4_data_unification/Player%20Golden%20ID%20of%20Football%20Datasets.ipynb).

___

<a id='sectioncontents'></a>

## <a id='notebook_contents'>Notebook Contents</a>
1.    [Notebook Dependencies](#section1)<br>
2.    [Project Brief](#section2)<br>
3.    [Data Sources](#section3)<br>
4.    [Data Engineering](#section4)<br>
      1.    [Introduction](#section3.1)<br>
      2.    [Outfielder Players](#section3.2)<br>
            1.    [Data Dictionary](#section3.2.1)<br>
            2.    [Creating the DataFrame](#section3.2.2)<br>
            3.    [Initial Data Handling](#section3.2.3)<br>
            4.    [Export the Raw DataFrame](#section3.2.4)<br>
5.    [Data Unification](#section5)<br>
6.    ['Golden ID' Table Creation](#section6)<br>
6.    [Data Export](#section6)<br>
7.    [Summary](#section7)<br>
8.    [Next Steps](#section8)<br>
9.    [Bibliography](#section9)<br>

___

<a id='section1'></a>

## <a id='#section1'>1. Notebook Dependencies</a>
This notebook was written using [Python 3](https://docs.python.org/3.7/) and requires the following libraries:
*    [`Jupyter notebooks`](https://jupyter.org/) for this notebook environment with which this project is presented;
*    [`NumPy`](http://www.numpy.org/) for multidimensional array computing;
*    [`pandas`](http://pandas.pydata.org/) for data analysis and manipulation; and
*    [`RecordLinkage`](https://pypi.org/project/recordlinkage/) for joining of fuzzy datasets.

All packages used for this notebook except for BeautifulSoup can be obtained by downloading and installing the [Conda](https://anaconda.org/anaconda/conda) distribution, available on all platforms (Windows, Linux and Mac OSX). Step-by-step guides on how to install Anaconda can be found for Windows [here](https://medium.com/@GalarnykMichael/install-python-on-windows-anaconda-c63c7c3d1444) and Mac [here](https://medium.com/@GalarnykMichael/install-python-on-mac-anaconda-ccd9f2014072), as well as in the Anaconda documentation itself [here](https://docs.anaconda.com/anaconda/install/).

### Import Libraries and Modules

In [1]:
# Python ≥3.5 (ideally)
import platform
import sys, getopt
assert sys.version_info >= (3, 5)
import csv

# Import Dependencies
%matplotlib inline

# Math Operations
import numpy as np
from math import pi

# Datetime
import datetime
from datetime import date
import time

# Data Preprocessing
import pandas as pd
#import pandas_profiling as pp
import os
import re
import chardet
import random
from io import BytesIO
from pathlib import Path

# Reading Directories
import glob
import os

# Working with JSON
import json
from pandas.io.json import json_normalize

# Web Scraping
import requests
from bs4 import BeautifulSoup
import re

# Fuzzy Matching - Record Linkage
import recordlinkage
import jellyfish
import numexpr as ne

# Data Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-whitegrid')
import missingno as msno

# Progress Bar
from tqdm import tqdm

# Display in Jupyter
from IPython.display import Image, YouTubeVideo
from IPython.core.display import HTML

# Ignore Warnings
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

print("Setup Complete")

Setup Complete


In [2]:
# Python / module versions used here for reference
print('Python: {}'.format(platform.python_version()))
print('NumPy: {}'.format(np.__version__))
print('pandas: {}'.format(pd.__version__))
print('matplotlib: {}'.format(mpl.__version__))

Python: 3.7.6
NumPy: 1.20.3
pandas: 1.3.2
matplotlib: 3.4.2


### Defined Variables

In [3]:
# Define today's date
today = datetime.datetime.now().strftime('%d/%m/%Y').replace('/', '')

### Defined Dictionaries

In [4]:
# Define seasons
dict_seasons = {'2016-2017': '2016/2017',
                '2017-2018': '2017/2018',
                '2018-2019': '2018/2019',
                '2019-2020': '2019/2020',
                '2020-2021': '2020/2021',
                '2021-2022': '2021/2022'
               }

### Define Data Paths

In [5]:
# Set up initial paths to subfolders
base_dir = os.path.join('..', '..')
data_dir = os.path.join(base_dir, 'data')
data_dir_fbref = os.path.join(base_dir, 'data', 'fbref')
data_dir_tm = os.path.join(base_dir, 'data', 'tm')
data_dir_capology = os.path.join(base_dir, 'data', 'capology')
img_dir = os.path.join(base_dir, 'img')
fig_dir = os.path.join(base_dir, 'img', 'fig')
video_dir = os.path.join(base_dir, 'video')

### Notebook Settings

In [6]:
pd.set_option('display.max_columns', None)

---

<a id='section2'></a>

## <a id='#section2'>2. Project Brief</a>

### <a id='#section2.1'>2.1. About this notebook</a>
This Jupyter notebook is part of a series of notebooks to scrape, parse, engineer, unify, and then model, culminating in a an Expected Transfer (xTransfer) player performance vs. valuation model. This model aims to determine the under- and over-performing players based on their on-the-pitch output against transfer fee and wages.

This particular notebook is the **golden id** notebook, that creates a reference table of all the different datasets, can then by used to join datasets together.

This notebook, along with the other notebooks in this project workflow are shown in the following diagram:

![roadmap](../../img/football_analytics_data_roadmap.png)

Links to these notebooks in the [`football_analytics`](https://github.com/eddwebster/football_analytics) GitHub repository can be found at the following:
*    [Webscraping](https://github.com/eddwebster/football_analytics/tree/master/notebooks/1_data_scraping)
     +    [FBref Player Stats Webscraping](https://github.com/eddwebster/football_analytics/blob/master/notebooks/1_data_scraping/FBref%20Player%20Stats%20Web%20Scraping.ipynb)
     +    [TransferMarket Player Bio and Status Webscraping](https://github.com/eddwebster/football_analytics/blob/master/notebooks/1_data_scraping/TransferMarkt%20Player%20Bio%20and%20Status%20Web%20Scraping.ipynb)
     +    [TransferMarkt Player Recorded Transfer Fees Webscraping](https://github.com/eddwebster/football_analytics/blob/master/notebooks/1_data_scraping/TransferMarkt%20Player%20Recorded%20Transfer%20Fees%20Webscraping.ipynb)
     +    [Capology Player Salary Webscraping](https://github.com/eddwebster/football_analytics/blob/master/notebooks/1_data_scraping/Capology%20Player%20Salary%20Web%20Scraping.ipynb)
     +    [FBref Team Stats Webscraping](https://github.com/eddwebster/football_analytics/blob/master/notebooks/1_data_scraping/FBref%20Team%20Stats%20Web%20Scraping.ipynb)
*    [Data Parsing](https://github.com/eddwebster/football_analytics/tree/master/notebooks/2_data_parsing)
     +    [ELO Team Ratings Data Parsing](https://github.com/eddwebster/football_analytics/blob/master/notebooks/2_data_parsing/ELO%20Team%20Ratings%20Data%20Parsing.ipynb)
*    [Data Engineering](https://github.com/eddwebster/football_analytics/tree/master/notebooks/3_data_engineering)
     +    [FBref Player Stats Data Engineering](https://github.com/eddwebster/football_analytics/blob/master/notebooks/3_data_engineering/FBref%20Player%20Stats%20Data%20Engineering.ipynb)
     +    [TransferMarket Player Bio and Status Data Engineering](https://github.com/eddwebster/football_analytics/blob/master/notebooks/3_data_engineering/TransferMarkt%20Player%20Bio%20and%20Status%20Data%20Engineering.ipynb)
     +    [TransferMarkt Player Recorded Transfer Fees Data Engineering](https://github.com/eddwebster/football_analytics/blob/master/notebooks/3_data_engineering/TransferMarkt%20Player%20Recorded%20Transfer%20Fees%20Data%20Engineering.ipynb)
     +    [Capology Player Salary Data Engineering](https://github.com/eddwebster/football_analytics/blob/master/notebooks/3_data_engineering/Capology%20Player%20Salary%20Data%20Engineering.ipynb)
     +    [FBref Team Stats Data Engineering](https://github.com/eddwebster/football_analytics/blob/master/notebooks/3_data_engineering/FBref%20Team%20Stats%20Data%20Engineering.ipynb)
     +    [ELO Team Ratings Data Parsing](https://github.com/eddwebster/football_analytics/blob/master/notebooks/3_data_engineering/ELO%20Team%20Ratings%20Data%20Parsing.ipynb)
     +    [TransferMarkt Team Recorded Transfer Fee Data Engineering](https://github.com/eddwebster/football_analytics/blob/master/notebooks/3_data_engineering/TransferMarkt%20Team%20Recorded%20Transfer%20Fee%20Data%20Engineering.ipynb) (aggregated from [TransferMarkt Player Recorded Transfer Fees notebook](https://github.com/eddwebster/football_analytics/blob/master/notebooks/3_data_engineering/TransferMarkt%20Player%20Recorded%20Transfer%20Fees%20Data%20Engineering.ipynb))
     +    [Capology Team Salary Data Engineering](https://github.com/eddwebster/football_analytics/blob/master/notebooks/3_data_engineering/Capology%20Team%20Salary%20Data%20Engineering.ipynb) (aggregated from [Capology Player Salary notebook](https://github.com/eddwebster/football_analytics/blob/master/notebooks/3_data_engineering/Capology%20Player%20Salary%20Data%20Engineering.ipynb))
*    [Data Unification](https://github.com/eddwebster/football_analytics/tree/master/notebooks/4_data_unification)
     +    [Golden ID for Player Level Datasets](https://github.com/eddwebster/football_analytics/blob/master/notebooks/4_data_unification/Golden%20ID%20for%20Player%20Level%20Datasets.ipynb)
     +    [Golden ID for Team Level Datasets](https://github.com/eddwebster/football_analytics/blob/master/notebooks/4_data_unification/Golden%20ID%20for%20Team%20Level%20Datasets.ipynb)
*    [Production Datasets](https://github.com/eddwebster/football_analytics/tree/master/notebooks/5_production_datasets)
     +    [Player Performance/Market Value Dataset](https://github.com/eddwebster/football_analytics/tree/master/notebooks/5_production_datasets/Player%20Performance/Market%20Value%20Dataset.ipynb)
     +    [Team Performance/Market Value Dataset](https://github.com/eddwebster/football_analytics/tree/master/notebooks/5_production_datasets/Team%20Performance/Market%20Value%20Dataset.ipynb)
*    [Expected Transfer (xTransfer) Modeling](https://github.com/eddwebster/football_analytics/tree/master/notebooks/6_data_analysis_and_projects/expected_transfer_modeling)
     +    [Expected Transfer (xTransfer) Modeling](https://github.com/eddwebster/football_analytics/tree/master/notebooks/6_data_analysis_and_projects/expected_transfer_modeling/Expected%20Transfer%20%20Modeling.ipynb)

**Notebook Conventions**:<br>
*    Variables that refer a `DataFrame` object are prefixed with `df_`.
*    Variables that refer to a collection of `DataFrame` objects (e.g., a list, a set or a dict) are prefixed with `dfs_`.

This Jupyter notebook matches disparate datasets from [FBref](https://fbref.com/en/) (provided by [StatsBomb](https://statsbomb.com/)), and [TransferMarkt](https://www.transfermarkt.co.uk/), and [Capology](https://www.capology.com/), using [RecordLinkage](https://pypi.org/project/recordlinkage/) and [pandas](http://pandas.pydata.org/) for data maniuplation through DataFrames.

Depending on the dataset, the data is joined on a player level or team level. This is reflecting in two output files - for players and teams.

---

<a id='section3'></a>

## <a id='#section3'>3. Data Sources</a>

### <a id='#section3.1'>3.1. Introduction</a>
The datasets include:
- ...
- ...
- ...
- ...

### <a id='#section3.2'>3.2. Read in CSV files as pandas DataFrames</a>
The following cells read in the `CSV` files as a pandas `DataFrame`s

In [7]:
# Import data as a pandas DataFrames

## FBref-TransferMarkt Player Mapping (including page URLs)

### Define file location
file = data_dir + '/reference/player_mapping/fbref_to_tm_mapping_latest.csv'

###
with open(file, 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))

### Read in dataset
df_fbref_tm_urls = pd.read_csv(file, encoding='ISO-8859-1')


## FBRef Player Performance data
df_fbref_players = pd.read_csv(data_dir_fbref + '/engineered/outfield-goalkeeper-combined/fbref_outfield_player_goalkeeper_stats_combined_latest.csv')


## TransferMarkt Player Bio-Status data
df_tm_bio_status = pd.read_csv(data_dir_tm + '/engineered/bio-status/tm_player_bio_status_all_1617-2122_latest.csv')


## TransferMarkt Player Historical Market Values data
df_tm_valuations = pd.read_csv(data_dir_tm + '/engineered/historical_market_values/tm_player_valuations_all_1617-2122_latest.csv')


## TransferMarkt Player Recorded Transfer History data
df_tm_transfers = pd.read_csv(data_dir_tm + '/engineered/transfer_history/tm_player_transfer_history_latest.csv')


## Capology Player Salary data
df_capology = pd.read_csv(data_dir_capology + '/engineered/capology_big5_mls_latest.csv')

<a id='section3.3'></a>

### <a id='#section3.3'>3.3. Initial Data Handling</a>

#### FBref to TM Player Mapping
Source: https://github.com/JaseZiv/worldfootballR_data/blob/master/raw-data/fbref-tm-player-mapping/output/fbref_to_tm_mapping.csv

In [8]:
df_fbref_tm_urls.head()

,PlayerFBref,UrlFBref,UrlTmarkt
0,Ãmer Toprak,https://fbref.com/en/players/c5fd13e9/Omer-Toprak,https://www.transfermarkt.com/omer-toprak/prof...
1,Å ime Vrsaljko,https://fbref.com/en/players/46a8347f/Sime-Vrs...,https://www.transfermarkt.com/sime-vrsaljko/pr...
2,Å imon Å tefanec,https://fbref.com/en/players/dff4a9fc/Simon-St...,https://www.transfermarkt.com/simon-stefanec/p...
3,Ãrjan Nyland,https://fbref.com/en/players/bdedffac/Orjan-Ny...,https://www.transfermarkt.com/orjan-nyland/pro...
4,Ãliver Torres,https://fbref.com/en/players/58a9a4d9/Oliver-T...,https://www.transfermarkt.com/oliver-torres/pr...


In [9]:
df_fbref_tm_urls.shape

(6167, 3)

#### FBref Players

In [10]:
df_fbref_players.head()

,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,Min,90s,Gls,Ast,G-PK,PK,PKatt,CrdY,CrdR,Gls.1,Ast.1,G+A,G-PK.1,G+A-PK,xG,npxG,xA,npxG+xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA.1,Matches,Sh,SoT,SoT%,Sh/90,SoT/90,G/Sh,G/SoT,Dist,FK,npxG/Sh,G-xG,np:G-xG,Cmp,Att,Cmp%,TotDist,PrgDist,Cmp.1,Att.1,Cmp%.1,Cmp.2,Att.2,Cmp%.2,Cmp.3,Att.3,Cmp%.3,A-xA,KP,1/3,PPA,CrsPA,Prog,Live,Dead,TB,Press,Sw,Crs,CK,In,Out,Str,Ground,Low,High,Left,Right,Head,TI,Other,Off,Out.1,Int,Blocks,SCA,SCA90,PassLive,PassDead,Drib,Fld,Def,GCA,GCA90,PassLive.1,PassDead.1,Drib.1,Sh.1,Fld.1,Def.1,Tkl,TklW,Def 3rd,Mid 3rd,Att 3rd,Tkl.1,Tkl%,Past,Succ,%,Def 3rd.1,Mid 3rd.1,Att 3rd.1,ShSv,Pass,Tkl+Int,Clr,Err,Touches,Def Pen,Att Pen,Succ%,#Pl,Megs,Carries,CPA,Mis,Dis,Targ,Rec,Rec%,Prog.1,Mn/MP,Min%,Mn/Start,Compl,Subs,Mn/Sub,unSub,PPM,onG,onGA,+/-,+/-90,On-Off,onxG,onxGA,xG+/-,xG+/-90,On-Off.1,2CrdY,Fls,PKwon,PKcon,OG,Recov,Won,Lost,Won%,League Name,League ID,Season,Team Name,Team Country,Player Lower,First Name Lower,Last Name Lower,First Initial Lower,Team Country Lower,Nationality Code,Nationality Cleaned,Position Grouped,Outfielder Goalkeeper,GA,GA90,SoTA,Saves,Save%,W,D,L,CS,CS%,PKA,PKsv,PKm,Save%.1,PSxG,PSxG/SoT,PSxG+/-,/90,Thr,Launch%,AvgLen,Launch%.1,AvgLen.1,Opp,Stp,Stp%,#OPA,#OPA/90,AvgDist
0,Aaron Cresswell,eng ENG,DF,West Ham,eng Premier League,27,1989.0,36,35,3069.0,34.1,1,3,1,0,0,7,0,0.03,0.09,0.12,0.03,0.12,0.8,0.8,2.8,3.6,0.02,0.08,0.10,0.02,0.10,Matches,21.0,6.0,28.6,0.62,0.18,0.05,0.17,28.1,8.0,0.04,0.2,0.2,1224.0,1708.0,71.7,23519.0,10212.0,560.0,623.0,89.9,472.0,587.0,80.4,183.0,449.0,40.8,0.2,35.0,117.0,21.0,14.0,96.0,1343.0,365.0,1.0,222.0,83.0,93.0,67.0,35.0,15.0,9.0,893.0,293.0,522.0,1329.0,78.0,59.0,210.0,5.0,15.0,44.0,39.0,52.0,62.0,1.82,35.0,21.0,1.0,3.0,0.0,9.0,0.26,6.0,3.0,0.0,0.0,0.0,0.0,38.0,18.0,15.0,18.0,5.0,17.0,53.1,15.0,115.0,32.1,181.0,123.0,54.0,0.0,38.0,90.0,133.0,0.0,2050.0,125.0,17.0,33.3,7.0,0.0,1071.0,2.0,18.0,19.0,1171.0,1094.0,93.4,31.0,85,89.7,NaN,30.0,1,NaN,1,1.14,45.0,60.0,-15.0,-0.44,0.84,38.0,51.5,-13.5,-0.40,1.09,0.0,20,0.0,0.0,0.0,277.0,70.0,57.0,55.1,Big-5-European-Leagues,Big5,2017-2018,West Ham,England,aaron cresswell,aaron,cresswell,a,england,ENG,England,Defender,Outfielder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aaron Hunt,de GER,"MF,FW",Hamburger SV,de Bundesliga,30,1986.0,28,26,2081.0,23.1,3,2,2,1,1,1,0,0.13,0.09,0.22,0.09,0.17,2.8,2.1,5.6,7.6,0.12,0.23,0.35,0.09,0.32,Matches,27.0,6.0,22.2,1.17,0.26,0.07,0.33,23.4,10.0,0.08,0.2,-0.1,883.0,1229.0,71.8,16889.0,5315.0,406.0,480.0,84.6,292.0,376.0,77.7,165.0,303.0,54.5,-3.6,65.0,83.0,31.0,5.0,97.0,977.0,252.0,11.0,245.0,67.0,66.0,123.0,35.0,41.0,14.0,672.0,236.0,321.0,999.0,137.0,42.0,23.0,9.0,5.0,29.0,29.0,49.0,102.0,4.25,54.0,43.0,1.0,2.0,1.0,6.0,0.25,5.0,1.0,0.0,0.0,0.0,0.0,30.0,22.0,12.0,16.0,2.0,5.0,13.5,32.0,135.0,27.9,102.0,261.0,121.0,0.0,28.0,44.0,21.0,0.0,1475.0,28.0,68.0,58.3,23.0,4.0,892.0,7.0,45.0,42.0,1176.0,893.0,75.9,178.0,74,68.0,NaN,14.0,2,NaN,0,1.07,22.0,34.0,-12.0,-0.52,0.58,27.0,31.3,-4.3,-0.18,0.94,0.0,27,0.0,0.0,0.0,213.0,22.0,37.0,37.3,Big-5-European-Leagues,Big5,2017-2018,Hamburger SV,Germany,aaron hunt,aaron,hunt,a,germany,GER,Germany,Midfielder,Outfielder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Aaron Lennon,eng ENG,MF,Burnley,eng Premier League,30,1987.0,14,13,1118.0,12.4,0,2,0,0,0,2,0,0.00,0.16,0.16,0.00,0.16,0.6,0.6,1.4,2.0,0.05,0.11,0.16,0.05,0.16,Matches,10.0,4.0,40.0,0.81,0.32,0.00,0.00,16.6,0.0,0.06,-0.6,-0.6,204.0,294.0,69.4,3223.0,887.0,116.0,142.0,81.7,68.0,92.0,73.9,17.0,34.0,50.0,0.6,8.0,11.0,13.0,5.0,22.0,289.0,5.0,0.0,61.0,5.0,19.0,0.0,0.0,0.0,0.0,193.0,51.0,50.0,27.0,250.0,7.0,4.0,3.0,0.0,9.0,8.0,30.0,18.0,1.45,12.0,0.0,1.0,1.0,0.0,3.0,0.24,2.0,0.0,0.0,1.0,0.0,0.0,18.0,10.0,6.0,11.0,1.0,4.0,19.0,17.0,61.0,26.3,74.0,102.0,56.0,0.0,24.0,31.0,9.0,0.0,424.0,19.0,36.0,48.0,12.0,2.0,29

In [11]:
df_fbref_players.shape

(13680, 204)

#### TransferMarkt Bio and Status
Player bio and status data

In [12]:
df_tm_bio_status.head()

,tm_id,player_name,birth_day,birth_month,birth_year,pob,cob,dob,position,height,foot,citizenship,second_citizenship,league_code,season,current_club,current_club_country,market_value_euros,joined,contract_expires,contract_option,on_loan_from,on_loan_from_country,loan_contract_expiry,player_agent,name_lower,firstname_lower,lastname_lower,firstinitial_lower,league_country_lower,position_code,position_grouped,outfielder_goalkeeper,age,age_when_joining,years_since_joining,years_until_contract_expiry,market_value_pounds
0,2857,eldin jakupovic,2.0,10.0,1984.0,Kozarac,Jugoslawien (SFR),1984-10-02,Goalkeeper,191.0,right,NaN,Bosnia-Herzegovina,GB1,2021,leicester city,england,300000.0,2017-07-19,2021-06-30,NaN,NaN,NaN,NaN,HSD,eldin jakupovic,eldin,jakupovic,e,england,GK,Goalkeeper,Goalkeeper,36.0,32.0,4.0,-1.0,270000.0
1,3333,james milner,4.0,1.0,1986.0,Leeds,England,1986-01-04,midfield - Central Midfield,175.0,right,England,NaN,GB1,2021,liverpool fc,england,3000000.0,2015-07-01,2022-06-30,NaN,NaN,NaN,NaN,Samii Sport-Marketing Agentur,james milner,james,milner,j,england,CM,Midfielder,Outfielder,35.0,29.0,6.0,0.0,2700000.0
2,3455,zlatan ibrahimovic,3.0,10.0,1981.0,Malmö,Sweden,1981-10-03,attack - Centre-Forward,195.0,both,NaN,Bosnia-Herzegovina,IT1,2021,ac milan,italy,4000000.0,2020-01-02,2022-06-30,NaN,NaN,NaN,NaN,Mino Raiola,zlatan ibrahimovic,zlatan,ibrahimovic,z,italy,ST,Forward,Outfielder,39.0,38.0,1.0,0.0,3600000.0
3,5578,nicolas penneteau,28.0,2.0,1981.0,Marseille,France,1981-02-28,Goalkeeper,185.0,left,France,NaN,FR1,2021,stade reims,france,200000.0,2021-07-01,2023-06-30,NaN,NaN,NaN,NaN,USM GROUP,nicolas penneteau,nicolas,penneteau,n,france,GK,Goalkeeper,Goalkeeper,40.0,40.0,0.0,1.0,180000.0
4,6442,antonio rosati,26.0,6.0,1983.0,Tivoli,Italy,1983-06-26,Goalkeeper,195.0,right,Italy,NaN,IT1,2021,acf fiorentina,italy,100000.0,2021-02-01,NaN,NaN,NaN,NaN,NaN,Alessandro Lucci - WSA,antonio rosati,antonio,rosati,a,italy,GK,Goalkeeper,Goalkeeper,38.0,37.0,0.0,NaN,90000.0


In [13]:
df_tm_bio_status.shape

(9429, 38)

#### TransferMarkt Historical Player Valuations
Historical player valuation data

In [14]:
df_tm_valuations.head()

,tm_id,season,player_name,club,current_club,league_code,current_age,market_value_gbp,market_value_eur,dob,pob,birth_year,position,position_code,position_grouped,outfielder_goalkeeper,height,foot,citizenship,second_citizenship,player_agent
0,26,2004/2005,roman weidenfeller,Borussia Dortmund,retired,L1,41.0,1800000.0,2000000,1980-08-06,Diez,1980.0,Goalkeeper,GK,Goalkeeper,Goalkeeper,188.0,left,Germany,NaN,Jörg Neubauer
1,26,2005/2006,roman weidenfeller,Borussia Dortmund,retired,L1,41.0,6075000.0,6750000,1980-08-06,Diez,1980.0,Goalkeeper,GK,Goalkeeper,Goalkeeper,188.0,left,Germany,NaN,Jörg Neubauer
2,26,2006/2007,roman weidenfeller,Borussia Dortmund,retired,L1,41.0,6750000.0,7500000,1980-08-06,Diez,1980.0,Goalkeeper,GK,Goalkeeper,Goalkeeper,188.0,left,Germany,NaN,Jörg Neubauer
3,26,2007/2008,roman weidenfeller,Borussia Dortmund,retired,L1,41.0,7200000.0,8000000,1980-08-06,Diez,1980.0,Goalkeeper,GK,Goalkeeper,Goalkeeper,188.0,left,Germany,NaN,Jörg Neubauer
4,26,2008/2009,roman weidenfeller,Borussia Dortmund,retired,L1,41.0,4500000.0,5000000,1980-08-06,Diez,1980.0,Goalkeeper,GK,Goalkeeper,Goalkeeper,188.0,left,Germany,NaN,Jörg Neubauer


In [15]:
df_tm_valuations.shape

(67236, 21)

#### TransferMarkt Recorded Transfers
Player recorded transfer data

In [16]:
df_tm_transfers.head()

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,year,season,league_code
0,VfB Stuttgart,Adrian Knup,23.0,Centre-Forward,FC Luzern,?,in,Summer,NaN,1 Bundesliga,1992,1992/1993,L1
1,1. FC Köln,Adrian Spyrka,24.0,Central Midfield,Stuttg. Kickers,"End of loanJun 30, 1992",in,Summer,0.0,1 Bundesliga,1992,1992/1993,L1
2,Karlsruher SC,Alexander Famulla,31.0,Goalkeeper,FC 08 Homburg,?,out,Summer,NaN,1 Bundesliga,1992,1992/1993,L1
3,SV Werder Bremen,Alexander Malchow,22.0,Centre-Back,VfB Oldenburg,Free transfer,out,Summer,0.0,1 Bundesliga,1992,1992/1993,L1
4,SG Dynamo Dresden,Alexander Zickler,18.0,Centre-Forward,D. Dresden U19,-,in,Summer,0.0,1 Bundesliga,1992,1992/1993,L1


In [17]:
df_tm_transfers.shape

(169208, 13)

#### Capology Player Salaries
Player salaries

In [18]:
df_capology.head()

,player,season,league,team,position,outfielder_goalkeeper,age,country,weekly_gross_base_salary_gbp,annual_gross_base_salary_gbp,adj_current_gross_base_salary_gbp,estimated_gross_total_gbp,current_contract_status,current_contract_expiration,current_contract_length
0,Albian Ajeti,2016-2017,Bundesliga,Augsburg,Forward,Outfielder,19,Switzerland,0.0,0.0,0.0,NaN,NaN,NaN,NaN
1,Alexander Esswein,2016-2017,Bundesliga,Augsburg,Forward,Outfielder,26,Germany,12919.0,671795.0,696824.0,NaN,NaN,NaN,NaN
2,Alfred Finnbogason,2016-2017,Bundesliga,Augsburg,Forward,Outfielder,27,Iceland,0.0,0.0,0.0,NaN,NaN,NaN,NaN
3,Andreas Luthe,2016-2017,Bundesliga,Augsburg,Goalkeeper,Goalkeeper,29,Germany,5939.0,308881.0,320389.0,NaN,NaN,NaN,NaN
4,Caiuby,2016-2017,Bundesliga,Augsburg,Forward,Outfielder,27,Brazil,12919.0,671795.0,696824.0,NaN,NaN,NaN,NaN


In [19]:
df_capology.shape

(21281, 15)

---

<a id='section4'></a>

## <a id='#section4'>4. Data Engineering</a>

<a id='section4.1'></a>

### <a id='#section4.1'>4.1. FBref-TransferMarkt Mapping (URLs)

##### Extract TransferMarkt ID
Last six digits of TransferMarkt URL: https://www.transfermarkt.com/jack-grealish/profil/spieler/203460.

In [20]:
df_fbref_tm_urls['tm_id'] = df_fbref_tm_urls['UrlTmarkt'].str.rsplit('/', n=1).str.get(-1)

##### Extract FBref ID
Penultimate eight digits of FBref URL: https://fbref.com/en/players/b0b4fd3e/Jack-Grealish

In [21]:
df_fbref_tm_urls['fbref_id'] = df_fbref_tm_urls['UrlFBref'].str.rsplit('/', n=1).str.get(-2)
df_fbref_tm_urls['fbref_id'] = df_fbref_tm_urls['fbref_id'].str.rsplit('/', n=1).str.get(-1)

##### Rename columns

In [22]:
## Rename columns
df_fbref_tm_urls = (df_fbref_tm_urls
                         .rename(columns={'PlayerFBref': 'player_name_fbref',
                                          'UrlFBref': 'url_fbref',
                                          'UrlTmarkt': 'url_tm'
                                         }
                                )
                    )

In [23]:
df_fbref_tm_urls.head()

,player_name_fbref,url_fbref,url_tm,tm_id,fbref_id
0,Ãmer Toprak,https://fbref.com/en/players/c5fd13e9/Omer-Toprak,https://www.transfermarkt.com/omer-toprak/prof...,43512,c5fd13e9
1,Å ime Vrsaljko,https://fbref.com/en/players/46a8347f/Sime-Vrs...,https://www.transfermarkt.com/sime-vrsaljko/pr...,76061,46a8347f
2,Å imon Å tefanec,https://fbref.com/en/players/dff4a9fc/Simon-St...,https://www.transfermarkt.com/simon-stefanec/p...,344802,dff4a9fc
3,Ãrjan Nyland,https://fbref.com/en/players/bdedffac/Orjan-Ny...,https://www.transfermarkt.com/orjan-nyland/pro...,73517,bdedffac
4,Ãliver Torres,https://fbref.com/en/players/58a9a4d9/Oliver-T...,https://www.transfermarkt.com/oliver-torres/pr...,214775,58a9a4d9


<a id='section4.2'></a>

### <a id='#section4.1'>4.2. FBref Players

##### Filter for 'Big 5' European Leagues

In [24]:
lst_leagues_fbref_players = list(df_fbref_players['Team Country'].unique())
lst_leagues_fbref_players

['England', 'Germany', 'Spain', 'France', 'Italy']

In [25]:
lst_leagues_fbref_players_big5 = ['England', 'Germany', 'Spain', 'France', 'Italy']

In [26]:
df_fbref_players = df_fbref_players[df_fbref_players['Team Country'].isin(lst_leagues_fbref_players_big5)]

##### Filter for Seasons

In [27]:
# Map season to DataFrame
df_fbref_players['Season'] = df_fbref_players['Season'].map(dict_seasons)

In [28]:
lst_seasons_fbref_players = list(df_fbref_players['Season'].unique())
lst_seasons_fbref_players

['2017/2018', '2018/2019', '2019/2020', '2020/2021', '2021/2022']

In [29]:
lst_seasons_fbref_players = ['2017/2018', '2018/2019', '2019/2020', '2020/2021', '2021/2022']

In [30]:
df_fbref_players = df_fbref_players[df_fbref_players['Season'].isin(lst_seasons_fbref_players)]

##### Lower names

In [31]:
# Remove accents and create lowercase name
df_fbref_players['player_name_lower'] = (df_fbref_players['Player']
                                             .str.normalize('NFKD')
                                             .str.encode('ascii', errors='ignore')
                                             .str.decode('utf-8')
                                             .str.lower()
                                        )

In [32]:
# First Name Lower
df_fbref_players['first_name_lower'] = df_fbref_players['player_name_lower'].str.rsplit(' ', 0).str[0]

# Last Name Lower
df_fbref_players['last_name_lower'] = df_fbref_players['player_name_lower'].str.rsplit(' ', 1).str[-1]

# First Initial Lower
df_fbref_players['first_initial_lower'] = df_fbref_players['player_name_lower'].astype(str).str[0]

In [33]:
# Remove accents and create lowercase name
df_fbref_players['country_lower'] = (df_fbref_players['Nationality Cleaned']
                                         .str.normalize('NFKD')
                                         .str.encode('ascii', errors='ignore')
                                         .str.decode('utf-8')
                                         .str.lower()
                                    )

In [34]:
# Display DataFrame
df_fbref_players.head()

,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,Min,90s,Gls,Ast,G-PK,PK,PKatt,CrdY,CrdR,Gls.1,Ast.1,G+A,G-PK.1,G+A-PK,xG,npxG,xA,npxG+xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA.1,Matches,Sh,SoT,SoT%,Sh/90,SoT/90,G/Sh,G/SoT,Dist,FK,npxG/Sh,G-xG,np:G-xG,Cmp,Att,Cmp%,TotDist,PrgDist,Cmp.1,Att.1,Cmp%.1,Cmp.2,Att.2,Cmp%.2,Cmp.3,Att.3,Cmp%.3,A-xA,KP,1/3,PPA,CrsPA,Prog,Live,Dead,TB,Press,Sw,Crs,CK,In,Out,Str,Ground,Low,High,Left,Right,Head,TI,Other,Off,Out.1,Int,Blocks,SCA,SCA90,PassLive,PassDead,Drib,Fld,Def,GCA,GCA90,PassLive.1,PassDead.1,Drib.1,Sh.1,Fld.1,Def.1,Tkl,TklW,Def 3rd,Mid 3rd,Att 3rd,Tkl.1,Tkl%,Past,Succ,%,Def 3rd.1,Mid 3rd.1,Att 3rd.1,ShSv,Pass,Tkl+Int,Clr,Err,Touches,Def Pen,Att Pen,Succ%,#Pl,Megs,Carries,CPA,Mis,Dis,Targ,Rec,Rec%,Prog.1,Mn/MP,Min%,Mn/Start,Compl,Subs,Mn/Sub,unSub,PPM,onG,onGA,+/-,+/-90,On-Off,onxG,onxGA,xG+/-,xG+/-90,On-Off.1,2CrdY,Fls,PKwon,PKcon,OG,Recov,Won,Lost,Won%,League Name,League ID,Season,Team Name,Team Country,Player Lower,First Name Lower,Last Name Lower,First Initial Lower,Team Country Lower,Nationality Code,Nationality Cleaned,Position Grouped,Outfielder Goalkeeper,GA,GA90,SoTA,Saves,Save%,W,D,L,CS,CS%,PKA,PKsv,PKm,Save%.1,PSxG,PSxG/SoT,PSxG+/-,/90,Thr,Launch%,AvgLen,Launch%.1,AvgLen.1,Opp,Stp,Stp%,#OPA,#OPA/90,AvgDist,player_name_lower,first_name_lower,last_name_lower,first_initial_lower,country_lower
0,Aaron Cresswell,eng ENG,DF,West Ham,eng Premier League,27,1989.0,36,35,3069.0,34.1,1,3,1,0,0,7,0,0.03,0.09,0.12,0.03,0.12,0.8,0.8,2.8,3.6,0.02,0.08,0.10,0.02,0.10,Matches,21.0,6.0,28.6,0.62,0.18,0.05,0.17,28.1,8.0,0.04,0.2,0.2,1224.0,1708.0,71.7,23519.0,10212.0,560.0,623.0,89.9,472.0,587.0,80.4,183.0,449.0,40.8,0.2,35.0,117.0,21.0,14.0,96.0,1343.0,365.0,1.0,222.0,83.0,93.0,67.0,35.0,15.0,9.0,893.0,293.0,522.0,1329.0,78.0,59.0,210.0,5.0,15.0,44.0,39.0,52.0,62.0,1.82,35.0,21.0,1.0,3.0,0.0,9.0,0.26,6.0,3.0,0.0,0.0,0.0,0.0,38.0,18.0,15.0,18.0,5.0,17.0,53.1,15.0,115.0,32.1,181.0,123.0,54.0,0.0,38.0,90.0,133.0,0.0,2050.0,125.0,17.0,33.3,7.0,0.0,1071.0,2.0,18.0,19.0,1171.0,1094.0,93.4,31.0,85,89.7,NaN,30.0,1,NaN,1,1.14,45.0,60.0,-15.0,-0.44,0.84,38.0,51.5,-13.5,-0.40,1.09,0.0,20,0.0,0.0,0.0,277.0,70.0,57.0,55.1,Big-5-European-Leagues,Big5,2017/2018,West Ham,England,aaron cresswell,aaron,cresswell,a,england,ENG,England,Defender,Outfielder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,aaron cresswell,aaron,cresswell,a,england
1,Aaron Hunt,de GER,"MF,FW",Hamburger SV,de Bundesliga,30,1986.0,28,26,2081.0,23.1,3,2,2,1,1,1,0,0.13,0.09,0.22,0.09,0.17,2.8,2.1,5.6,7.6,0.12,0.23,0.35,0.09,0.32,Matches,27.0,6.0,22.2,1.17,0.26,0.07,0.33,23.4,10.0,0.08,0.2,-0.1,883.0,1229.0,71.8,16889.0,5315.0,406.0,480.0,84.6,292.0,376.0,77.7,165.0,303.0,54.5,-3.6,65.0,83.0,31.0,5.0,97.0,977.0,252.0,11.0,245.0,67.0,66.0,123.0,35.0,41.0,14.0,672.0,236.0,321.0,999.0,137.0,42.0,23.0,9.0,5.0,29.0,29.0,49.0,102.0,4.25,54.0,43.0,1.0,2.0,1.0,6.0,0.25,5.0,1.0,0.0,0.0,0.0,0.0,30.0,22.0,12.0,16.0,2.0,5.0,13.5,32.0,135.0,27.9,102.0,261.0,121.0,0.0,28.0,44.0,21.0,0.0,1475.0,28.0,68.0,58.3,23.0,4.0,892.0,7.0,45.0,42.0,1176.0,893.0,75.9,178.0,74,68.0,NaN,14.0,2,NaN,0,1.07,22.0,34.0,-12.0,-0.52,0.58,27.0,31.3,-4.3,-0.18,0.94,0.0,27,0.0,0.0,0.0,213.0,22.0,37.0,37.3,Big-5-European-Leagues,Big5,2017/2018,Hamburger SV,Germany,aaron hunt,aaron,hunt,a,germany,GER,Germany,Midfielder,Outfielder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,aaron hunt,aaron,hunt,a,germany
2,Aaron Lennon,eng ENG,MF,Burnley,eng Premier League,30,1987.0,14,13,1118.0,12.4,0,2,0,0,0,2,0,0.00,0.16,0.16,0.00,0.16,0.6,0.6,1.4,2.0,0.05,0.11,0.16,0.05,0.16,Matches,10.0,4.0,40.0,0.81,0.32,0.00,0.00,16.6,0.0,0.06,-0.6,-0.6,204.0,294.0,69.4,3223.0,887.0,116.0,142.0,81.7,68.0,92.0,73.9,17.0,34.0,50.0,0.6,8.0,11.0,13.0,5.0,22.0,289.0,5.0,0.0,61.0,5.0,19.0,0.0,0.0,0.0,0.0,193.0,51.0,50.0,27.0,250.0,7.0,4.0,3.0,0.0,9.0,8.0,30.0,18.0,1.45,12.0,0.0,1.0

##### Rename columns

In [35]:
## Rename columns
df_fbref_players = (df_fbref_players
                         .rename(columns={'Born': 'birth_year',
                                          'Outfielder Goalkeeper': 'outfielder_goalkeeper',
                                          'Season': 'season',
                                         }
                                )
                    )

##### Select columns of interest

In [36]:
## Define columns
cols_fbref_players = ['Player',
                      'first_initial_lower',
                      'first_name_lower',
                      'last_name_lower',
                     #'age',
                      'birth_year',
                      'country_lower',
                      'outfielder_goalkeeper',
                      'season'
                     ]

## Select columns of interest
df_fbref_players_select = df_fbref_players[cols_fbref_players]

# Drop duplicates
df_fbref_players_select = df_fbref_players_select.drop_duplicates()

# Display DataFrame
df_fbref_players_select.head()

,Player,first_initial_lower,first_name_lower,last_name_lower,birth_year,country_lower,outfielder_goalkeeper,season
0,Aaron Cresswell,a,aaron,cresswell,1989.0,england,Outfielder,2017/2018
1,Aaron Hunt,a,aaron,hunt,1986.0,germany,Outfielder,2017/2018
2,Aaron Lennon,a,aaron,lennon,1987.0,england,Outfielder,2017/2018
4,Aaron Mooy,a,aaron,mooy,1990.0,australia,Outfielder,2017/2018
5,Aaron Ramsey,a,aaron,ramsey,1990.0,wales,Outfielder,2017/2018


<a id='section4.3'></a>

### <a id='#section4.3'>4.3. TransferMarkt Bio and Status

##### Filter for 'Big 5' European Leagues

In [37]:
lst_leagues_tm_bio_status = list(df_tm_bio_status['league_code'].unique())
lst_leagues_tm_bio_status

['GB1', 'IT1', 'FR1', 'L1', 'ES1', 'MLS1']

In [38]:
lst_leagues_tm_bio_status_big5 = ['GB1', 'IT1', 'FR1', 'L1', 'ES1']

In [39]:
df_tm_bio_status = df_tm_bio_status[df_tm_bio_status['league_code'].isin(lst_leagues_tm_bio_status_big5)]

##### Lower names

In [40]:
# Remove accents and create lowercase name
df_tm_bio_status['player_name_lower'] = (df_tm_bio_status['player_name']
                                             .str.normalize('NFKD')
                                             .str.encode('ascii', errors='ignore')
                                             .str.decode('utf-8')
                                             .str.lower()
                                        )

In [41]:
# First Name Lower
df_tm_bio_status['first_name_lower'] = df_tm_bio_status['player_name_lower'].str.rsplit(' ', 0).str[0]

# Last Name Lower
df_tm_bio_status['last_name_lower'] = df_tm_bio_status['player_name_lower'].str.rsplit(' ', 1).str[-1]

# First Initial Lower
df_tm_bio_status['first_initial_lower'] = df_tm_bio_status['player_name_lower'].astype(str).str[0]

In [42]:
# Remove accents and create lowercase name
df_tm_bio_status['country_lower'] = (df_tm_bio_status['cob']
                                         .str.normalize('NFKD')
                                         .str.encode('ascii', errors='ignore')
                                         .str.decode('utf-8')
                                         .str.lower()
                                    )

In [43]:
# Display DataFrame
df_tm_bio_status.head()

,tm_id,player_name,birth_day,birth_month,birth_year,pob,cob,dob,position,height,foot,citizenship,second_citizenship,league_code,season,current_club,current_club_country,market_value_euros,joined,contract_expires,contract_option,on_loan_from,on_loan_from_country,loan_contract_expiry,player_agent,name_lower,firstname_lower,lastname_lower,firstinitial_lower,league_country_lower,position_code,position_grouped,outfielder_goalkeeper,age,age_when_joining,years_since_joining,years_until_contract_expiry,market_value_pounds,player_name_lower,first_name_lower,last_name_lower,first_initial_lower,country_lower
0,2857,eldin jakupovic,2.0,10.0,1984.0,Kozarac,Jugoslawien (SFR),1984-10-02,Goalkeeper,191.0,right,NaN,Bosnia-Herzegovina,GB1,2021,leicester city,england,300000.0,2017-07-19,2021-06-30,NaN,NaN,NaN,NaN,HSD,eldin jakupovic,eldin,jakupovic,e,england,GK,Goalkeeper,Goalkeeper,36.0,32.0,4.0,-1.0,270000.0,eldin jakupovic,eldin,jakupovic,e,jugoslawien (sfr)
1,3333,james milner,4.0,1.0,1986.0,Leeds,England,1986-01-04,midfield - Central Midfield,175.0,right,England,NaN,GB1,2021,liverpool fc,england,3000000.0,2015-07-01,2022-06-30,NaN,NaN,NaN,NaN,Samii Sport-Marketing Agentur,james milner,james,milner,j,england,CM,Midfielder,Outfielder,35.0,29.0,6.0,0.0,2700000.0,james milner,james,milner,j,england
2,3455,zlatan ibrahimovic,3.0,10.0,1981.0,Malmö,Sweden,1981-10-03,attack - Centre-Forward,195.0,both,NaN,Bosnia-Herzegovina,IT1,2021,ac milan,italy,4000000.0,2020-01-02,2022-06-30,NaN,NaN,NaN,NaN,Mino Raiola,zlatan ibrahimovic,zlatan,ibrahimovic,z,italy,ST,Forward,Outfielder,39.0,38.0,1.0,0.0,3600000.0,zlatan ibrahimovic,zlatan,ibrahimovic,z,sweden
3,5578,nicolas penneteau,28.0,2.0,1981.0,Marseille,France,1981-02-28,Goalkeeper,185.0,left,France,NaN,FR1,2021,stade reims,france,200000.0,2021-07-01,2023-06-30,NaN,NaN,NaN,NaN,USM GROUP,nicolas penneteau,nicolas,penneteau,n,france,GK,Goalkeeper,Goalkeeper,40.0,40.0,0.0,1.0,180000.0,nicolas penneteau,nicolas,penneteau,n,france
4,6442,antonio rosati,26.0,6.0,1983.0,Tivoli,Italy,1983-06-26,Goalkeeper,195.0,right,Italy,NaN,IT1,2021,acf fiorentina,italy,100000.0,2021-02-01,NaN,NaN,NaN,NaN,NaN,Alessandro Lucci - WSA,antonio rosati,antonio,rosati,a,italy,GK,Goalkeeper,Goalkeeper,38.0,37.0,0.0,NaN,90000.0,antonio rosati,antonio,rosati,a,italy


##### Select columns of interest

In [44]:
# Define columns
cols_tm_bio_status = ['player_name',
                      'first_initial_lower',
                      'first_name_lower',
                      'last_name_lower',
                     #'age',
                      'birth_year',
                      'country_lower',
                      'outfielder_goalkeeper',
                      'tm_id'
                     ]

# Select columns of interest
df_tm_bio_status_select = df_tm_bio_status[cols_tm_bio_status]

# Drop duplicates
df_tm_bio_status_select = df_tm_bio_status_select.drop_duplicates()

# Display DataFrame
df_tm_bio_status_select.head()

,player_name,first_initial_lower,first_name_lower,last_name_lower,birth_year,country_lower,outfielder_goalkeeper,tm_id
0,eldin jakupovic,e,eldin,jakupovic,1984.0,jugoslawien (sfr),Goalkeeper,2857
1,james milner,j,james,milner,1986.0,england,Outfielder,3333
2,zlatan ibrahimovic,z,zlatan,ibrahimovic,1981.0,sweden,Outfielder,3455
3,nicolas penneteau,n,nicolas,penneteau,1981.0,france,Goalkeeper,5578
4,antonio rosati,a,antonio,rosati,1983.0,italy,Goalkeeper,6442


<a id='section4.4'></a>

### <a id='#section4.4'>4.4. TransferMarkt Historical Player Valuations

##### Filter for 'Big 5' European Leagues

In [45]:
lst_leagues_tm_valuations = list(df_tm_valuations['league_code'].unique())
lst_leagues_tm_valuations

['L1', 'GB1', 'MLS1', 'FR1', 'ES1', 'IT1']

In [46]:
lst_leagues_tm_bio_status_big5 = ['GB1', 'IT1', 'FR1', 'L1', 'ES1']

In [47]:
df_tm_valuations = df_tm_valuations[df_tm_valuations['league_code'].isin(lst_leagues_tm_bio_status_big5)]

##### Filter for Seasons

In [48]:
lst_seasons_tm_valuations = list(df_tm_valuations['season'].unique())
lst_seasons_tm_valuations

['2004/2005',
 '2005/2006',
 '2006/2007',
 '2007/2008',
 '2008/2009',
 '2010/2011',
 '2011/2012',
 '2012/2013',
 '2013/2014',
 '2014/2015',
 '2015/2016',
 '2016/2017',
 '2017/2018',
 '2018/2019',
 '2009/2010',
 '2019/2020',
 '2020/2021',
 '2021/2022']

In [49]:
lst_seasons_tm_valuations = ['2017/2018', '2018/2019', '2019/2020', '2020/2021', '2021/2022']

In [50]:
df_tm_valuations = df_tm_valuations[df_tm_valuations['season'].isin(lst_seasons_tm_valuations)]

##### Lower names

In [51]:
# Remove accents and create lowercase name
df_tm_valuations['player_name_lower'] = (df_tm_valuations['player_name']
                                             .str.normalize('NFKD')
                                             .str.encode('ascii', errors='ignore')
                                             .str.decode('utf-8')
                                             .str.lower()
                                        )

In [52]:
# First Name Lower
df_tm_valuations['first_name_lower'] = df_tm_valuations['player_name_lower'].str.rsplit(' ', 0).str[0]

# Last Name Lower
df_tm_valuations['last_name_lower'] = df_tm_valuations['player_name_lower'].str.rsplit(' ', 1).str[-1]

# First Initial Lower
df_tm_valuations['first_initial_lower'] = df_tm_valuations['player_name_lower'].astype(str).str[0]

In [53]:
# Remove accents and create lowercase name
df_tm_valuations['country_lower'] = (df_tm_valuations['citizenship']
                                         .str.normalize('NFKD')
                                         .str.encode('ascii', errors='ignore')
                                         .str.decode('utf-8')
                                         .str.lower()
                                    )

In [54]:
# Display DataFrame
df_tm_valuations.head()

,tm_id,season,player_name,club,current_club,league_code,current_age,market_value_gbp,market_value_eur,dob,pob,birth_year,position,position_code,position_grouped,outfielder_goalkeeper,height,foot,citizenship,second_citizenship,player_agent,player_name_lower,first_name_lower,last_name_lower,first_initial_lower,country_lower
12,26,2017/2018,roman weidenfeller,Borussia Dortmund,retired,L1,41.0,675000.0,750000,1980-08-06,Diez,1980.0,Goalkeeper,GK,Goalkeeper,Goalkeeper,188.0,left,Germany,NaN,Jörg Neubauer,roman weidenfeller,roman,weidenfeller,r,germany
13,26,2018/2019,roman weidenfeller,Borussia Dortmund,retired,L1,41.0,0.0,0,1980-08-06,Diez,1980.0,Goalkeeper,GK,Goalkeeper,Goalkeeper,188.0,left,Germany,NaN,Jörg Neubauer,roman weidenfeller,roman,weidenfeller,r,germany
27,80,2017/2018,tom starke,Bayern Munich,retired,L1,40.0,90000.0,100000,1981-03-18,Freital,1981.0,Goalkeeper,GK,Goalkeeper,Goalkeeper,194.0,right,Germany,NaN,IFM,tom starke,tom,starke,t,germany
28,80,2018/2019,tom starke,Bayern Munich,retired,L1,40.0,90000.0,100000,1981-03-18,Freital,1981.0,Goalkeeper,GK,Goalkeeper,Goalkeeper,194.0,right,Germany,NaN,IFM,tom starke,tom,starke,t,germany
41,488,2017/2018,gerhard tremmel,Swansea City,retired,GB1,42.0,225000.0,250000,1978-11-16,München,1978.0,Goalkeeper,GK,Goalkeeper,Goalkeeper,NaN,NaN,Germany,NaN,NaN,gerhard tremmel,gerhard,tremmel,g,germany


##### Select columns of interest

In [55]:
# Define columns
cols_tm_valuations = ['player_name',
                      'first_initial_lower',
                      'first_name_lower',
                      'last_name_lower',
                     #'age',
                      'birth_year',
                      'country_lower',
                      'outfielder_goalkeeper',
                      'season',
                      'tm_id'
                     ]

# Select columns of interest
df_tm_valuations_select = df_tm_valuations[cols_tm_valuations]

# Drop duplicates
df_tm_valuations_select = df_tm_valuations_select.drop_duplicates()

# Display DataFrame
df_tm_valuations_select.head()

,player_name,first_initial_lower,first_name_lower,last_name_lower,birth_year,country_lower,outfielder_goalkeeper,season,tm_id
12,roman weidenfeller,r,roman,weidenfeller,1980.0,germany,Goalkeeper,2017/2018,26
13,roman weidenfeller,r,roman,weidenfeller,1980.0,germany,Goalkeeper,2018/2019,26
27,tom starke,t,tom,starke,1981.0,germany,Goalkeeper,2017/2018,80
28,tom starke,t,tom,starke,1981.0,germany,Goalkeeper,2018/2019,80
41,gerhard tremmel,g,gerhard,tremmel,1978.0,germany,Goalkeeper,2017/2018,488


<a id='section4.5'></a>

### <a id='#section4.5'>4.5. TransferMarkt Recorded Transfers

##### Filter for 'Big 5' European Leagues

In [56]:
lst_leagues_tm_transfers = list(df_tm_transfers['league_code'].unique())
lst_leagues_tm_transfers

['L1', 'GB2', 'NL1', 'PO1', 'FR1', 'GB1', 'RU1', 'ES1', 'IT1']

In [57]:
lst_leagues_tm_transfers_big5 = ['GB1', 'IT1', 'FR1', 'L1', 'ES1']

In [58]:
df_tm_valuations = df_tm_valuations[df_tm_valuations['league_code'].isin(lst_leagues_tm_transfers_big5)]

##### Filter for Seasons

In [59]:
lst_seasons_tm_transfers = list(df_tm_transfers['season'].unique())
lst_seasons_tm_transfers

['1992/1993',
 '1993/1994',
 '1994/1995',
 '1995/1996',
 '1996/1997',
 '1997/1998',
 '1998/1999',
 '1999/2000',
 '2000/2001',
 '2001/2002',
 '2002/2003',
 '2003/2004',
 '2004/2005',
 '2005/2006',
 '2006/2007',
 '2007/2008',
 '2008/2009',
 '2009/2010',
 '2010/2011',
 '2011/2012',
 '2012/2013',
 '2013/2014',
 '2014/2015',
 '2015/2016',
 '2016/2017',
 '2017/2018',
 '2018/2019',
 '2019/2020',
 '2020/2021']

In [60]:
lst_leagues_tm_transfers_big5 = ['2017/2018', '2018/2019', '2019/2020', '2020/2021', '2021/2022']

In [61]:
df_tm_valuations = df_tm_valuations[df_tm_valuations['season'].isin(lst_leagues_tm_transfers_big5)]

##### Lower names

In [62]:
# Remove accents and create lowercase name
df_tm_transfers['player_name_lower'] = (df_tm_transfers['player_name']
                                            .str.normalize('NFKD')
                                            .str.encode('ascii', errors='ignore')
                                            .str.decode('utf-8')
                                            .str.lower()
                                       )

In [63]:
# First Name Lower
df_tm_transfers['first_name_lower'] = df_tm_transfers['player_name_lower'].str.rsplit(' ', 0).str[0]

# Last Name Lower
df_tm_transfers['last_name_lower'] = df_tm_transfers['player_name_lower'].str.rsplit(' ', 1).str[-1]

# First Initial Lower
df_tm_transfers['first_initial_lower'] = df_tm_transfers['player_name_lower'].astype(str).str[0]

In [64]:
# Display DataFrame
df_tm_transfers.head()

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,year,season,league_code,player_name_lower,first_name_lower,last_name_lower,first_initial_lower
0,VfB Stuttgart,Adrian Knup,23.0,Centre-Forward,FC Luzern,?,in,Summer,NaN,1 Bundesliga,1992,1992/1993,L1,adrian knup,adrian,knup,a
1,1. FC Köln,Adrian Spyrka,24.0,Central Midfield,Stuttg. Kickers,"End of loanJun 30, 1992",in,Summer,0.0,1 Bundesliga,1992,1992/1993,L1,adrian spyrka,adrian,spyrka,a
2,Karlsruher SC,Alexander Famulla,31.0,Goalkeeper,FC 08 Homburg,?,out,Summer,NaN,1 Bundesliga,1992,1992/1993,L1,alexander famulla,alexander,famulla,a
3,SV Werder Bremen,Alexander Malchow,22.0,Centre-Back,VfB Oldenburg,Free transfer,out,Summer,0.0,1 Bundesliga,1992,1992/1993,L1,alexander malchow,alexander,malchow,a
4,SG Dynamo Dresden,Alexander Zickler,18.0,Centre-Forward,D. Dresden U19,-,in,Summer,0.0,1 Bundesliga,1992,1992/1993,L1,alexander zickler,alexander,zickler,a


##### Select columns of interest

In [65]:
# Define columns
cols_tm_transfers = ['player_name',
                     'first_initial_lower',
                     'first_name_lower',
                     'last_name_lower',
                     'season'
                    ]

# Select columns of interest
df_tm_transfers_select = df_tm_transfers[cols_tm_transfers]

# Drop duplicates
df_tm_transfers_select = df_tm_transfers.drop_duplicates()

# Display DataFrame
df_tm_transfers_select.head()

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,year,season,league_code,player_name_lower,first_name_lower,last_name_lower,first_initial_lower
0,VfB Stuttgart,Adrian Knup,23.0,Centre-Forward,FC Luzern,?,in,Summer,NaN,1 Bundesliga,1992,1992/1993,L1,adrian knup,adrian,knup,a
1,1. FC Köln,Adrian Spyrka,24.0,Central Midfield,Stuttg. Kickers,"End of loanJun 30, 1992",in,Summer,0.0,1 Bundesliga,1992,1992/1993,L1,adrian spyrka,adrian,spyrka,a
2,Karlsruher SC,Alexander Famulla,31.0,Goalkeeper,FC 08 Homburg,?,out,Summer,NaN,1 Bundesliga,1992,1992/1993,L1,alexander famulla,alexander,famulla,a
3,SV Werder Bremen,Alexander Malchow,22.0,Centre-Back,VfB Oldenburg,Free transfer,out,Summer,0.0,1 Bundesliga,1992,1992/1993,L1,alexander malchow,alexander,malchow,a
4,SG Dynamo Dresden,Alexander Zickler,18.0,Centre-Forward,D. Dresden U19,-,in,Summer,0.0,1 Bundesliga,1992,1992/1993,L1,alexander zickler,alexander,zickler,a


<a id='section4.6'></a>

### <a id='#section4.6'>4.6. Capology Player Salaries

##### Filter for 'Big 5' European Leagues

In [66]:
lst_leagues_capology = list(df_capology['country'].unique())
lst_leagues_capology

['Switzerland',
 'Germany',
 'Iceland',
 'Brazil',
 'Ghana',
 'South Korea',
 'Austria',
 'Serbia',
 'Turkey',
 'Greece',
 'Czech Republic',
 'Netherlands',
 'France',
 'Paraguay',
 'Japan',
 'Slovenia',
 'Latvia',
 'Chile',
 'Mexico',
 'Finland',
 'Jamaica',
 'Croatia',
 'Australia',
 'Ukraine',
 'Spain',
 'United States',
 'Morocco',
 'Portugal',
 'Poland',
 'Colombia',
 'Sweden',
 'Denmark',
 'Gabon',
 'Costa Rica',
 'Hungary',
 'Bosnia-Herzegovina',
 'Nigeria',
 'Democratic Republic of Congo',
 'Tunisia',
 'Ecuador',
 'Argentina',
 'Kosovo',
 'Uruguay',
 'Israel',
 'Albania',
 'Palästina',
 'Norway',
 'Slovakia',
 'Georgia',
 'Italy',
 'The Gambia',
 "Cote d'Ivoire",
 'Cameroon',
 'Russia',
 'Philippines',
 'Belgium',
 'Guinea',
 'Scotland',
 'Bulgaria',
 'Algeria',
 'Peru',
 'Senegal',
 'Uganda',
 'Mali',
 'Iran',
 'Venezuela',
 'England',
 'Azerbaijan',
 'Curacao',
 'Togo',
 'Montenegro',
 'Romania',
 'China',
 'Canada',
 'Luxembourg',
 'Wales',
 'Burkina Faso',
 'New Zealand',
 

In [67]:
lst_leagues_capology_big5 = ['England', 'Germany', 'Spain', 'France', 'Italy']

In [68]:
df_capology = df_capology[df_capology['country'].isin(lst_leagues_capology_big5)]

##### Filter for Seasons

In [69]:
# Map season to DataFrame
df_capology['season'] = df_capology['season'].map(dict_seasons)

In [70]:
lst_seasons_capology = list(df_capology['season'].unique())
lst_seasons_capology

['2016/2017', '2017/2018', '2018/2019', '2019/2020', nan, '2020/2021']

In [71]:
lst_seasons_capology = ['2017/2018', '2018/2019', '2019/2020', '2020/2021', '2021/2022']

In [72]:
df_capology = df_capology[df_capology['season'].isin(lst_seasons_capology)]

##### Lower names

In [73]:
# Remove accents and create lowercase name
df_capology['player_name_lower'] = (df_capology['player']
                                        .str.normalize('NFKD')
                                        .str.encode('ascii', errors='ignore')
                                        .str.decode('utf-8')
                                        .str.lower()
                                   )

In [74]:
# First Name Lower
df_capology['first_name_lower'] = df_capology['player_name_lower'].str.rsplit(' ', 0).str[0]

# Last Name Lower
df_capology['last_name_lower'] = df_capology['player_name_lower'].str.rsplit(' ', 1).str[-1]

# First Initial Lower
df_capology['first_initial_lower'] = df_capology['player_name_lower'].astype(str).str[0]

In [75]:
# Remove accents and create lowercase name
df_capology['country_lower'] = (df_capology['country']
                                    .str.normalize('NFKD')
                                    .str.encode('ascii', errors='ignore')
                                    .str.decode('utf-8')
                                    .str.lower()
                               )

In [76]:
# Display DataFrame
df_capology.head()

,player,season,league,team,position,outfielder_goalkeeper,age,country,weekly_gross_base_salary_gbp,annual_gross_base_salary_gbp,adj_current_gross_base_salary_gbp,estimated_gross_total_gbp,current_contract_status,current_contract_expiration,current_contract_length,player_name_lower,first_name_lower,last_name_lower,first_initial_lower,country_lower
645,Andreas Luthe,2017/2018,Bundesliga,Augsburg,Goalkeeper,Goalkeeper,30,Germany,6147.0,319688.0,325953.0,NaN,NaN,NaN,NaN,andreas luthe,andreas,luthe,a,germany
647,Christoph Janker,2017/2018,Bundesliga,Augsburg,Defender,Outfielder,32,Germany,7585.0,394432.0,402161.0,NaN,NaN,NaN,NaN,christoph janker,christoph,janker,c,germany
648,Daniel Baier,2017/2018,Bundesliga,Augsburg,Midfielder,Outfielder,33,Germany,34185.0,1777646.0,1812482.0,NaN,NaN,NaN,NaN,daniel baier,daniel,baier,d,germany
651,Efkan Bekiroglu,2017/2018,Bundesliga,Augsburg,Midfielder,Outfielder,21,Germany,761.0,39623.0,40399.0,NaN,NaN,NaN,NaN,efkan bekiroglu,efkan,bekiroglu,e,germany
652,Erik Thommy,2017/2018,Bundesliga,Augsburg,Midfielder,Outfielder,22,Germany,7585.0,394432.0,402161.0,NaN,NaN,NaN,NaN,erik thommy,erik,thommy,e,germany


##### Select columns of interest

In [173]:
# Define columns
cols_capology = ['player',
                 'first_initial_lower',
                 'first_name_lower',
                 'last_name_lower',
                 'country_lower',
                 'outfielder_goalkeeper',
                 'season'
                ]

# Select columns of interest
df_capology_select = df_capology[cols_capology]

# Drop duplicates
df_capology_select = df_capology.drop_duplicates()

# Display DataFrame
df_capology_select.head()

,player,season,league,team,position,outfielder_goalkeeper,age,country,weekly_gross_base_salary_gbp,annual_gross_base_salary_gbp,adj_current_gross_base_salary_gbp,estimated_gross_total_gbp,current_contract_status,current_contract_expiration,current_contract_length,player_name_lower,first_name_lower,last_name_lower,first_initial_lower,country_lower
645,Andreas Luthe,2017/2018,Bundesliga,Augsburg,Goalkeeper,Goalkeeper,30,Germany,6147.0,319688.0,325953.0,NaN,NaN,NaN,NaN,andreas luthe,andreas,luthe,a,germany
647,Christoph Janker,2017/2018,Bundesliga,Augsburg,Defender,Outfielder,32,Germany,7585.0,394432.0,402161.0,NaN,NaN,NaN,NaN,christoph janker,christoph,janker,c,germany
648,Daniel Baier,2017/2018,Bundesliga,Augsburg,Midfielder,Outfielder,33,Germany,34185.0,1777646.0,1812482.0,NaN,NaN,NaN,NaN,daniel baier,daniel,baier,d,germany
651,Efkan Bekiroglu,2017/2018,Bundesliga,Augsburg,Midfielder,Outfielder,21,Germany,761.0,39623.0,40399.0,NaN,NaN,NaN,NaN,efkan bekiroglu,efkan,bekiroglu,e,germany
652,Erik Thommy,2017/2018,Bundesliga,Augsburg,Midfielder,Outfielder,22,Germany,7585.0,394432.0,402161.0,NaN,NaN,NaN,NaN,erik thommy,erik,thommy,e,germany


---

<a id='section5'></a>

## <a id='#section5'>5. Fuzzy Match Datasets - Record Linkage</a>

<a id='section5.1'></a>

### <a id='#section5.1'>5.1. Introduction</a>
Now we have the player-level datasets, we are now read to merge them to form one dataset of identifiers, that can then be used join any datasets together in future analysis.

We are required to join two data sets together that do not have a common unique identifier and are required to use a third-party Python library - [record linkage](https://pypi.org/project/recordlinkage/), installed using `pip install recordlinkage`. [record linkage](https://pypi.org/project/recordlinkage/) provides a simple interface to link records in or between data sources. 

As part of the joining of the datasets, the **FBref** dataset will be used as the base, from which the other datasets will be subsequently joined.

<a id='section5.2'></a>

### <a id='#section5.2'>5.2. About Record Linkage</a>

<b>'Record linkage'</b> is the term used by statisticians, epidemiologists, and historians, among others, to describe the process of joining records from one data source with another that describe the same entity ([source](https://en.wikipedia.org/wiki/Record_linkage)).

Record linkage is necessary when joining different data sets based on entities that may or may not share a common identifier (e.g., database key, URI, National identification number), which may be due to differences in record shape, storage location, or curator style or preference. A data set that has undergone RL-oriented reconciliation may be referred to as being cross-linked. Record linkage is referred to as data linkage in many jurisdictions, but the two are the same process.

The toolkit provides most of the tools needed for record linkage and deduplication. The package contains indexing methods, functions to compare records and classifiers. The package is developed for research and the linking of small or medium sized files.

For a full guide on how to use record linkage, see the official documentation [here](https://recordlinkage.readthedocs.io/en/latest/) and also this worked example by [Chris Moffitt](https://twitter.com/chris1610) at the following [[link](https://pbpython.com/record-linking.html)].

<a id='section5.3'></a>

### <a id='#section5.3'>5.3. Unify Datasets</a>

In [77]:
"""
# DataFrames
df_fbref_tm_urls_select
df_fbref_players_select
df_tm_bio_status_select
df_tm_valuations_select
df_tm_transfers_select
df_capology_select
"""

'\n# DataFrames\ndf_fbref_tm_urls_select\ndf_fbref_players_select\ndf_tm_bio_status_select\ndf_tm_valuations_select\ndf_tm_transfers_select\ndf_capology_select\n'

<a id='section5.3.1'></a>

#### <a id='#section5.3.1'>5.3.1. Join the FBref Players and FBref-TM mapping datasets</a>

In [125]:
# Join the Bio-Status dataset to the Historical Player Valuation dataset

## Join the TransferMarkt Bio-Status and Player Valuation DataFrames
df_fbref_merge = pd.merge(df_fbref_tm_urls, df_fbref_players, left_on='player_name_fbref', right_on='Player', how='left')


## Rename columns - required otherwise 'birth_year' gets dropped
df_fbref_merge = df_fbref_merge.rename(columns={'birth_year': 'born'})


## Remove duplicates

### Remove duplicate columns after join (contain '_y') and remove '_x' suffix from kept columns
df_fbref_merge = df_fbref_merge[df_fbref_merge.columns.drop(list(df_fbref_merge.filter(regex='_y')))]
df_fbref_merge.columns = df_fbref_merge.columns.str.replace('_x', '')

### Remove duplicate rows
df_fbref_merge = df_fbref_merge.drop_duplicates(subset=['season', 'player_name_fbref', 'Team Name', 'Team Country', 'Comp'], keep='first')


### Drop unnecessary columns
df_fbref_merge = df_fbref_merge.drop(['Player'], axis=1)


## Rename columns
df_fbref_merge = df_fbref_merge.rename(columns={'born': 'birth_year'})


## Sort columns
df_fbref_merge = df_fbref_merge.sort_values(by=['player_name_fbref'], ascending=[True])


## Display DataFrame
df_fbref_merge.head()

,player_name_fbref,url_fbref,url_tm,tm_id,fbref_id,Nation,Pos,Squad,Comp,Age,birth_year,MP,Starts,Min,90s,Gls,Ast,G-PK,PK,PKatt,CrdY,CrdR,Gls.1,Ast.1,G+A,G-PK.1,G+A-PK,xG,npxG,xA,npxG+xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA.1,Matches,Sh,SoT,SoT%,Sh/90,SoT/90,G/Sh,G/SoT,Dist,FK,npxG/Sh,G-xG,np:G-xG,Cmp,Att,Cmp%,TotDist,PrgDist,Cmp.1,Att.1,Cmp%.1,Cmp.2,Att.2,Cmp%.2,Cmp.3,Att.3,Cmp%.3,A-xA,KP,1/3,PPA,CrsPA,Prog,Live,Dead,TB,Press,Sw,Crs,CK,In,Out,Str,Ground,Low,High,Left,Right,Head,TI,Other,Off,Out.1,Int,Blocks,SCA,SCA90,PassLive,PassDead,Drib,Fld,Def,GCA,GCA90,PassLive.1,PassDead.1,Drib.1,Sh.1,Fld.1,Def.1,Tkl,TklW,Def 3rd,Mid 3rd,Att 3rd,Tkl.1,Tkl%,Past,Succ,%,Def 3rd.1,Mid 3rd.1,Att 3rd.1,ShSv,Pass,Tkl+Int,Clr,Err,Touches,Def Pen,Att Pen,Succ%,#Pl,Megs,Carries,CPA,Mis,Dis,Targ,Rec,Rec%,Prog.1,Mn/MP,Min%,Mn/Start,Compl,Subs,Mn/Sub,unSub,PPM,onG,onGA,+/-,+/-90,On-Off,onxG,onxGA,xG+/-,xG+/-90,On-Off.1,2CrdY,Fls,PKwon,PKcon,OG,Recov,Won,Lost,Won%,League Name,League ID,season,Team Name,Team Country,Player Lower,First Name Lower,Last Name Lower,First Initial Lower,Team Country Lower,Nationality Code,Nationality Cleaned,Position Grouped,outfielder_goalkeeper,GA,GA90,SoTA,Saves,Save%,W,D,L,CS,CS%,PKA,PKsv,PKm,Save%.1,PSxG,PSxG/SoT,PSxG+/-,/90,Thr,Launch%,AvgLen,Launch%.1,AvgLen.1,Opp,Stp,Stp%,#OPA,#OPA/90,AvgDist,player_name_lower,first_name_lower,last_name_lower,first_initial_lower,country_lower
240,Aaron Connolly,https://fbref.com/en/players/27c01749/Aaron-Co...,https://www.transfermarkt.com/aaron-connolly/p...,434207,27c01749,ie IRL,FW,Brighton,eng Premier League,19.0,2000.0,24.0,14.0,1258.0,14.0,3.0,1.0,3.0,0.0,0.0,0.0,0.0,0.21,0.07,0.29,0.21,0.29,3.2,3.2,0.3,3.5,0.23,0.02,0.25,0.23,0.25,Matches,38.0,13.0,34.2,2.72,0.93,0.08,0.23,15.9,0.0,0.08,-0.2,-0.2,126.0,163.0,77.3,1739.0,242.0,76.0,92.0,82.6,31.0,39.0,79.5,7.0,11.0,63.6,0.7,6.0,6.0,2.0,0.0,10.0,148.0,15.0,1.0,50.0,0.0,7.0,0.0,0.0,0.0,0.0,90.0,52.0,21.0,27.0,107.0,13.0,1.0,6.0,0.0,1.0,4.0,10.0,25.0,1.79,7.0,0.0,3.0,9.0,3.0,5.0,0.36,1.0,0.0,1.0,1.0,2.0,0.0,12.0,8.0,1.0,5.0,6.0,3.0,25.0,9.0,69.0,29.5,14.0,94.0,126.0,0.0,7.0,17.0,1.0,0.0,349.0,2.0,61.0,37.5,6.0,1.0,228.0,12.0,42.0,34.0,535.0,235.0,43.9,99.0,52.0,36.8,72.0,0.0,10.0,26.0,4.0,1.13,18.0,22.0,-4.0,-0.29,0.17,15.6,19.8,-4.2,-0.30,0.08,0.0,16.0,2.0,0.0,0.0,54.0,14.0,48.0,22.6,Big-5-European-Leagues,Big5,2019/2020,Brighton,England,aaron connolly,aaron,connolly,a,england,IRL,Ireland,Forward,Outfielder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,aaron connolly,aaron,connolly,a,ireland
241,Aaron Connolly,https://fbref.com/en/players/27c01749/Aaron-Co...,https://www.transfermarkt.com/aaron-connolly/p...,434207,27c01749,ie IRL,FW,Brighton,eng Premier League,20.0,2000.0,17.0,9.0,791.0,8.8,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.23,0.11,0.34,0.23,0.34,3.5,3.5,0.2,3.7,0.40,0.02,0.42,0.40,0.42,Matches,23.0,8.0,34.8,2.62,0.91,0.09,0.25,13.7,0.0,0.15,-1.5,-1.5,79.0,101.0,78.2,1147.0,165.0,45.0,56.0,80.4,26.0,30.0,86.7,4.0,5.0,80.0,0.8,5.0,2.0,1.0,0.0,3.0,91.0,10.0,0.0,22.0,1.0,2.0,0.0,0.0,0.0,0.0,64.0,26.0,11.0,11.0,74.0,5.0,0.0,3.0,0.0,0.0,2.0,4.0,12.0,1.37,7.0,0.0,3.0,2.0,0.0,1.0,0.11,1.0,0.0,0.0,0.0,0.0,0.0,7.0,5.0,2.0,4.0,1.0,1.0,20.0,4.0,40.0,32.3,7.0,58.0,59.0,0.0,8.0,7.0,1.0,0.0,201.0,1.0,38.0,80.0,8.0,0.0,124.0,4.0,29.0,15.0,357.0,143.0,40.1,64.0,47.0,23.1,68.0,NaN,8.0,23.0,11.0,0.88,12.0,17.0,-5.0,-0.57,-0.53,13.8,7.8,6.0,0.69,0.42,0.0,5.0,1.0,0.0,0.0,28.0,11.0,30.0,26.8,Big-5-European-Leagues,Big5,2020/2021,Brighton,England,aaron connolly,aaron,connolly,a,england,IRL,Ireland,Forward,Outfielder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,aaron connolly,aaron,connolly,a,ireland
242,Aaron Connolly,https://fbref.com/en/players/27c01749/Aaron-Co...,https://www.transfermarkt.com/aaron-connolly/p...,434207,27c01749,ie IRL,FW,Brighton,eng Premier League,21.0,2000.0,1.0,0.0,45.0,0.5,0.0,0.0,0.0,0.0,0.0,0.

In [126]:
print('No. rows in FBref Player DataFrame before join to FBref-TM Mapping data: {}'.format(len(df_fbref_players)))
print('No. rows in DataFrame AFTER join: {}\n'.format(len(df_fbref_merge)))
print('Variance in rows before and after join: {}\n'.format(len(df_fbref_merge) - len(df_fbref_players)))
print('-'*10)

No. rows in FBref Player DataFrame before join to FBref-TM Mapping data: 13680
No. rows in DataFrame AFTER join: 13986

Variance in rows before and after join: 306

----------


<a id='section5.3.2'></a>

#### <a id='#section5.3.2'>5.3.2. Join the three TransferMarkt Datasets</a>
- Bio-Status
- Historical Player Valuations
- Recorded Transfers

##### Join the Bio-Status to the Historical Player Valuation datasets

In [105]:
# Join the Bio-Status dataset to the Historical Player Valuation dataset

## Join the TransferMarkt Bio-Status and Player Valuation DataFrames
df_tm_merge = pd.merge(df_tm_valuations, df_tm_bio_status, left_on='player_name', right_on='player_name', how='left')


## Rename columns - required otherwise 'birth_year' gets dropped
df_tm_merge = df_tm_merge.rename(columns={'birth_year_x': 'born'})


## Remove duplicates

### Remove duplicate columns after join (contain '_y') and remove '_x' suffix from kept columns
df_tm_merge = df_tm_merge[df_tm_merge.columns.drop(list(df_tm_merge.filter(regex='_y')))]
df_tm_merge.columns = df_tm_merge.columns.str.replace('_x', '')

### Remove duplicate rows
df_tm_merge = df_tm_merge.drop_duplicates(subset=['tm_id', 'season', 'player_name'], keep='first')


## Rename columns
df_tm_merge = df_tm_merge.rename(columns={'born': 'birth_year'})


## Display DataFrame
df_tm_merge.head()

,tm_id,season,player_name,club,current_club,league_code,current_age,market_value_gbp,market_value_eur,dob,pob,birth_year,position,position_code,position_grouped,outfielder_goalkeeper,height,foot,citizenship,second_citizenship,player_agent,player_name_lower,first_name_lower,last_name_lower,first_initial_lower,country_lower,birth_day,birth_month,cob,current_club_country,market_value_euros,joined,contract_expires,contract_option,on_loan_from,on_loan_from_country,loan_contract_expiry,name_lower,firstname_lower,lastname_lower,firstinitial_lower,league_country_lower,age,age_when_joining,years_since_joining,years_until_contract_expiry,market_value_pounds
0,26,2017/2018,roman weidenfeller,Borussia Dortmund,retired,L1,41.0,675000.0,750000,1980-08-06,Diez,1980.0,Goalkeeper,GK,Goalkeeper,Goalkeeper,188.0,left,Germany,NaN,Jörg Neubauer,roman weidenfeller,roman,weidenfeller,r,germany,6.0,8.0,Germany,NaN,0.0,2018-07-01,NaN,NaN,NaN,NaN,NaN,roman weidenfeller,roman,weidenfeller,r,NaN,41.0,37.0,3.0,NaN,0.0
1,26,2018/2019,roman weidenfeller,Borussia Dortmund,retired,L1,41.0,0.0,0,1980-08-06,Diez,1980.0,Goalkeeper,GK,Goalkeeper,Goalkeeper,188.0,left,Germany,NaN,Jörg Neubauer,roman weidenfeller,roman,weidenfeller,r,germany,6.0,8.0,Germany,NaN,0.0,2018-07-01,NaN,NaN,NaN,NaN,NaN,roman weidenfeller,roman,weidenfeller,r,NaN,41.0,37.0,3.0,NaN,0.0
2,80,2017/2018,tom starke,Bayern Munich,retired,L1,40.0,90000.0,100000,1981-03-18,Freital,1981.0,Goalkeeper,GK,Goalkeeper,Goalkeeper,194.0,right,Germany,NaN,IFM,tom starke,tom,starke,t,germany,18.0,3.0,DDR,NaN,0.0,2018-07-01,NaN,NaN,NaN,NaN,NaN,tom starke,tom,starke,t,NaN,40.0,37.0,3.0,NaN,0.0
3,80,2018/2019,tom starke,Bayern Munich,retired,L1,40.0,90000.0,100000,1981-03-18,Freital,1981.0,Goalkeeper,GK,Goalkeeper,Goalkeeper,194.0,right,Germany,NaN,IFM,tom starke,tom,starke,t,germany,18.0,3.0,DDR,NaN,0.0,2018-07-01,NaN,NaN,NaN,NaN,NaN,tom starke,tom,starke,t,NaN,40.0,37.0,3.0,NaN,0.0
4,488,2017/2018,gerhard tremmel,Swansea City,retired,GB1,42.0,225000.0,250000,1978-11-16,München,1978.0,Goalkeeper,GK,Goalkeeper,Goalkeeper,NaN,NaN,Germany,NaN,NaN,gerhard tremmel,gerhard,tremmel,g,germany,16.0,11.0,Germany,NaN,0.0,2017-07-17,NaN,NaN,NaN,NaN,NaN,gerhard tremmel,gerhard,tremmel,g,NaN,42.0,38.0,4.0,NaN,0.0


In [85]:
print('No. rows in TM Player Valuation DataFrame before join to TransferMarkt Bio-Status data: {}'.format(len(df_tm_valuations)))
print('No. rows in DataFrame AFTER join: {}\n'.format(len(df_tm_merge)))
print('Variance in rows before and after join: {}\n'.format(len(df_tm_merge) - len(df_tm_valuations)))
print('-'*10)

No. rows in TM Player Valuation DataFrame before join to TransferMarkt Bio-Status data: 30241
No. rows in DataFrame AFTER join: 30241

Variance in rows before and after join: 0

----------


##### Join the Transfer dataset

In [86]:
# Join Player Transfer dataset to Bio-Status-Valuation dataset

## Join the TransferMarkt Bio-Status and Transfer DataFrames
df_tm_merge_final = pd.merge(df_tm_merge, df_tm_transfers, left_on=['player_name', 'season'], right_on=['player_name_lower', 'season'], how='left')


## Rename columns - required otherwise 'birth_year' gets dropped
df_tm_merge_final = df_tm_merge_final.rename(columns={'birth_year': 'born'})


## Remove duplicates

### Remove duplicate columns after join (contain '_y') and remove '_x' suffix from kept columns
df_tm_merge_final = df_tm_merge_final[df_tm_merge_final.columns.drop(list(df_tm_merge_final.filter(regex='_y')))]
df_tm_merge_final.columns = df_tm_merge_final.columns.str.replace('_x','')

### Remove duplicate rows
df_tm_merge_final = df_tm_merge_final.drop_duplicates(subset=['tm_id', 'season', 'player_name'], keep='first')


## Drop unnecessary columns
df_tm_merge_final = df_tm_merge_final.drop(['year'], axis=1)


## Rename columns
df_tm_merge_final = df_tm_merge_final.rename(columns={'born': 'birth_year'})


## Display DataFrame
df_tm_merge_final.head()

,tm_id,season,player_name,club,current_club,league_code,current_age,market_value_gbp,market_value_eur,dob,pob,birth_year,position,position_code,position_grouped,outfielder_goalkeeper,height,foot,citizenship,second_citizenship,player_agent,player_name_lower,first_name_lower,last_name_lower,first_initial_lower,country_lower,birth_day,birth_month,cob,current_club_country,market_value_euros,joined,contract_expires,contract_option,on_loan_from,on_loan_from_country,loan_contract_expiry,name_lower,firstname_lower,lastname_lower,firstinitial_lower,league_country_lower,age,age_when_joining,years_since_joining,years_until_contract_expiry,market_value_pounds,club_name,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name
0,26,2017/2018,roman weidenfeller,Borussia Dortmund,retired,L1,41.0,675000.0,750000,1980-08-06,Diez,1980.0,Goalkeeper,GK,Goalkeeper,Goalkeeper,188.0,left,Germany,NaN,Jörg Neubauer,roman weidenfeller,roman,weidenfeller,r,germany,6.0,8.0,Germany,NaN,0.0,2018-07-01,NaN,NaN,NaN,NaN,NaN,roman weidenfeller,roman,weidenfeller,r,NaN,41.0,37.0,3.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,26,2018/2019,roman weidenfeller,Borussia Dortmund,retired,L1,41.0,0.0,0,1980-08-06,Diez,1980.0,Goalkeeper,GK,Goalkeeper,Goalkeeper,188.0,left,Germany,NaN,Jörg Neubauer,roman weidenfeller,roman,weidenfeller,r,germany,6.0,8.0,Germany,NaN,0.0,2018-07-01,NaN,NaN,NaN,NaN,NaN,roman weidenfeller,roman,weidenfeller,r,NaN,41.0,37.0,3.0,NaN,0.0,Borussia Dortmund,Retired,-,out,Summer,0.0,1 Bundesliga
2,80,2017/2018,tom starke,Bayern Munich,retired,L1,40.0,90000.0,100000,1981-03-18,Freital,1981.0,Goalkeeper,GK,Goalkeeper,Goalkeeper,194.0,right,Germany,NaN,IFM,tom starke,tom,starke,t,germany,18.0,3.0,DDR,NaN,0.0,2018-07-01,NaN,NaN,NaN,NaN,NaN,tom starke,tom,starke,t,NaN,40.0,37.0,3.0,NaN,0.0,Bayern Munich,Career break,-,in,Summer,0.0,1 Bundesliga
4,80,2018/2019,tom starke,Bayern Munich,retired,L1,40.0,90000.0,100000,1981-03-18,Freital,1981.0,Goalkeeper,GK,Goalkeeper,Goalkeeper,194.0,right,Germany,NaN,IFM,tom starke,tom,starke,t,germany,18.0,3.0,DDR,NaN,0.0,2018-07-01,NaN,NaN,NaN,NaN,NaN,tom starke,tom,starke,t,NaN,40.0,37.0,3.0,NaN,0.0,Bayern Munich,Retired,-,out,Summer,0.0,1 Bundesliga
5,488,2017/2018,gerhard tremmel,Swansea City,retired,GB1,42.0,225000.0,250000,1978-11-16,München,1978.0,Goalkeeper,GK,Goalkeeper,Goalkeeper,NaN,NaN,Germany,NaN,NaN,gerhard tremmel,gerhard,tremmel,g,germany,16.0,11.0,Germany,NaN,0.0,2017-07-17,NaN,NaN,NaN,NaN,NaN,gerhard tremmel,gerhard,tremmel,g,NaN,42.0,38.0,4.0,NaN,0.0,Swansea City,Retired,-,out,Summer,0.0,Premier League


In [87]:
print('No. rows in merged TM Bio-Status-Valuation DataFrame before join to TM Recorded Transfer data: {}'.format(len(df_tm_merge)))
print('No. rows in DataFrame AFTER join: {}\n'.format(len(df_tm_merge_final)))
print('Variance in rows before and after join: {}\n'.format(len(df_tm_merge_final) - len(df_tm_merge)))
print('-'*10)

No. rows in merged TM Bio-Status-Valuation DataFrame before join to TM Recorded Transfer data: 30241
No. rows in DataFrame AFTER join: 30241

Variance in rows before and after join: 0

----------


<a id='section5.3.3'></a>

#### <a id='#section5.3.3'>5.3.3. Join the FBref and TM Datasets</a>
The first step is to create a indexer object.

The indexing module is used to make pairs of records. These pairs are called candidate links or candidate matches. There are several indexing algorithms available such as blocking and sorted neighborhood indexing. See the following references for background information about indexation

One key concept is that we can use blocking to limit the number of comparisons. For instance, we know that it is very likely that we only want to compare records that have the same last name, first name initial and birth year. We can use this knowledge to setup a block on these columns in both DataFrames:

In [127]:
df_fbref_merge.head()

,player_name_fbref,url_fbref,url_tm,tm_id,fbref_id,Nation,Pos,Squad,Comp,Age,birth_year,MP,Starts,Min,90s,Gls,Ast,G-PK,PK,PKatt,CrdY,CrdR,Gls.1,Ast.1,G+A,G-PK.1,G+A-PK,xG,npxG,xA,npxG+xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA.1,Matches,Sh,SoT,SoT%,Sh/90,SoT/90,G/Sh,G/SoT,Dist,FK,npxG/Sh,G-xG,np:G-xG,Cmp,Att,Cmp%,TotDist,PrgDist,Cmp.1,Att.1,Cmp%.1,Cmp.2,Att.2,Cmp%.2,Cmp.3,Att.3,Cmp%.3,A-xA,KP,1/3,PPA,CrsPA,Prog,Live,Dead,TB,Press,Sw,Crs,CK,In,Out,Str,Ground,Low,High,Left,Right,Head,TI,Other,Off,Out.1,Int,Blocks,SCA,SCA90,PassLive,PassDead,Drib,Fld,Def,GCA,GCA90,PassLive.1,PassDead.1,Drib.1,Sh.1,Fld.1,Def.1,Tkl,TklW,Def 3rd,Mid 3rd,Att 3rd,Tkl.1,Tkl%,Past,Succ,%,Def 3rd.1,Mid 3rd.1,Att 3rd.1,ShSv,Pass,Tkl+Int,Clr,Err,Touches,Def Pen,Att Pen,Succ%,#Pl,Megs,Carries,CPA,Mis,Dis,Targ,Rec,Rec%,Prog.1,Mn/MP,Min%,Mn/Start,Compl,Subs,Mn/Sub,unSub,PPM,onG,onGA,+/-,+/-90,On-Off,onxG,onxGA,xG+/-,xG+/-90,On-Off.1,2CrdY,Fls,PKwon,PKcon,OG,Recov,Won,Lost,Won%,League Name,League ID,season,Team Name,Team Country,Player Lower,First Name Lower,Last Name Lower,First Initial Lower,Team Country Lower,Nationality Code,Nationality Cleaned,Position Grouped,outfielder_goalkeeper,GA,GA90,SoTA,Saves,Save%,W,D,L,CS,CS%,PKA,PKsv,PKm,Save%.1,PSxG,PSxG/SoT,PSxG+/-,/90,Thr,Launch%,AvgLen,Launch%.1,AvgLen.1,Opp,Stp,Stp%,#OPA,#OPA/90,AvgDist,player_name_lower,first_name_lower,last_name_lower,first_initial_lower,country_lower
240,Aaron Connolly,https://fbref.com/en/players/27c01749/Aaron-Co...,https://www.transfermarkt.com/aaron-connolly/p...,434207,27c01749,ie IRL,FW,Brighton,eng Premier League,19.0,2000.0,24.0,14.0,1258.0,14.0,3.0,1.0,3.0,0.0,0.0,0.0,0.0,0.21,0.07,0.29,0.21,0.29,3.2,3.2,0.3,3.5,0.23,0.02,0.25,0.23,0.25,Matches,38.0,13.0,34.2,2.72,0.93,0.08,0.23,15.9,0.0,0.08,-0.2,-0.2,126.0,163.0,77.3,1739.0,242.0,76.0,92.0,82.6,31.0,39.0,79.5,7.0,11.0,63.6,0.7,6.0,6.0,2.0,0.0,10.0,148.0,15.0,1.0,50.0,0.0,7.0,0.0,0.0,0.0,0.0,90.0,52.0,21.0,27.0,107.0,13.0,1.0,6.0,0.0,1.0,4.0,10.0,25.0,1.79,7.0,0.0,3.0,9.0,3.0,5.0,0.36,1.0,0.0,1.0,1.0,2.0,0.0,12.0,8.0,1.0,5.0,6.0,3.0,25.0,9.0,69.0,29.5,14.0,94.0,126.0,0.0,7.0,17.0,1.0,0.0,349.0,2.0,61.0,37.5,6.0,1.0,228.0,12.0,42.0,34.0,535.0,235.0,43.9,99.0,52.0,36.8,72.0,0.0,10.0,26.0,4.0,1.13,18.0,22.0,-4.0,-0.29,0.17,15.6,19.8,-4.2,-0.30,0.08,0.0,16.0,2.0,0.0,0.0,54.0,14.0,48.0,22.6,Big-5-European-Leagues,Big5,2019/2020,Brighton,England,aaron connolly,aaron,connolly,a,england,IRL,Ireland,Forward,Outfielder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,aaron connolly,aaron,connolly,a,ireland
241,Aaron Connolly,https://fbref.com/en/players/27c01749/Aaron-Co...,https://www.transfermarkt.com/aaron-connolly/p...,434207,27c01749,ie IRL,FW,Brighton,eng Premier League,20.0,2000.0,17.0,9.0,791.0,8.8,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.23,0.11,0.34,0.23,0.34,3.5,3.5,0.2,3.7,0.40,0.02,0.42,0.40,0.42,Matches,23.0,8.0,34.8,2.62,0.91,0.09,0.25,13.7,0.0,0.15,-1.5,-1.5,79.0,101.0,78.2,1147.0,165.0,45.0,56.0,80.4,26.0,30.0,86.7,4.0,5.0,80.0,0.8,5.0,2.0,1.0,0.0,3.0,91.0,10.0,0.0,22.0,1.0,2.0,0.0,0.0,0.0,0.0,64.0,26.0,11.0,11.0,74.0,5.0,0.0,3.0,0.0,0.0,2.0,4.0,12.0,1.37,7.0,0.0,3.0,2.0,0.0,1.0,0.11,1.0,0.0,0.0,0.0,0.0,0.0,7.0,5.0,2.0,4.0,1.0,1.0,20.0,4.0,40.0,32.3,7.0,58.0,59.0,0.0,8.0,7.0,1.0,0.0,201.0,1.0,38.0,80.0,8.0,0.0,124.0,4.0,29.0,15.0,357.0,143.0,40.1,64.0,47.0,23.1,68.0,NaN,8.0,23.0,11.0,0.88,12.0,17.0,-5.0,-0.57,-0.53,13.8,7.8,6.0,0.69,0.42,0.0,5.0,1.0,0.0,0.0,28.0,11.0,30.0,26.8,Big-5-European-Leagues,Big5,2020/2021,Brighton,England,aaron connolly,aaron,connolly,a,england,IRL,Ireland,Forward,Outfielder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,aaron connolly,aaron,connolly,a,ireland
242,Aaron Connolly,https://fbref.com/en/players/27c01749/Aaron-Co...,https://www.transfermarkt.com/aaron-connolly/p...,434207,27c01749,ie IRL,FW,Brighton,eng Premier League,21.0,2000.0,1.0,0.0,45.0,0.5,0.0,0.0,0.0,0.0,0.0,0.

In [128]:
df_tm_merge_final.head()

,tm_id,season,player_name,club,current_club,league_code,current_age,market_value_gbp,market_value_eur,dob,pob,birth_year,position,position_code,position_grouped,outfielder_goalkeeper,height,foot,citizenship,second_citizenship,player_agent,player_name_lower,first_name_lower,last_name_lower,first_initial_lower,country_lower,birth_day,birth_month,cob,current_club_country,market_value_euros,joined,contract_expires,contract_option,on_loan_from,on_loan_from_country,loan_contract_expiry,name_lower,firstname_lower,lastname_lower,firstinitial_lower,league_country_lower,age,age_when_joining,years_since_joining,years_until_contract_expiry,market_value_pounds,club_name,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name
0,26,2017/2018,roman weidenfeller,Borussia Dortmund,retired,L1,41.0,675000.0,750000,1980-08-06,Diez,1980.0,Goalkeeper,GK,Goalkeeper,Goalkeeper,188.0,left,Germany,NaN,Jörg Neubauer,roman weidenfeller,roman,weidenfeller,r,germany,6.0,8.0,Germany,NaN,0.0,2018-07-01,NaN,NaN,NaN,NaN,NaN,roman weidenfeller,roman,weidenfeller,r,NaN,41.0,37.0,3.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,26,2018/2019,roman weidenfeller,Borussia Dortmund,retired,L1,41.0,0.0,0,1980-08-06,Diez,1980.0,Goalkeeper,GK,Goalkeeper,Goalkeeper,188.0,left,Germany,NaN,Jörg Neubauer,roman weidenfeller,roman,weidenfeller,r,germany,6.0,8.0,Germany,NaN,0.0,2018-07-01,NaN,NaN,NaN,NaN,NaN,roman weidenfeller,roman,weidenfeller,r,NaN,41.0,37.0,3.0,NaN,0.0,Borussia Dortmund,Retired,-,out,Summer,0.0,1 Bundesliga
2,80,2017/2018,tom starke,Bayern Munich,retired,L1,40.0,90000.0,100000,1981-03-18,Freital,1981.0,Goalkeeper,GK,Goalkeeper,Goalkeeper,194.0,right,Germany,NaN,IFM,tom starke,tom,starke,t,germany,18.0,3.0,DDR,NaN,0.0,2018-07-01,NaN,NaN,NaN,NaN,NaN,tom starke,tom,starke,t,NaN,40.0,37.0,3.0,NaN,0.0,Bayern Munich,Career break,-,in,Summer,0.0,1 Bundesliga
4,80,2018/2019,tom starke,Bayern Munich,retired,L1,40.0,90000.0,100000,1981-03-18,Freital,1981.0,Goalkeeper,GK,Goalkeeper,Goalkeeper,194.0,right,Germany,NaN,IFM,tom starke,tom,starke,t,germany,18.0,3.0,DDR,NaN,0.0,2018-07-01,NaN,NaN,NaN,NaN,NaN,tom starke,tom,starke,t,NaN,40.0,37.0,3.0,NaN,0.0,Bayern Munich,Retired,-,out,Summer,0.0,1 Bundesliga
5,488,2017/2018,gerhard tremmel,Swansea City,retired,GB1,42.0,225000.0,250000,1978-11-16,München,1978.0,Goalkeeper,GK,Goalkeeper,Goalkeeper,NaN,NaN,Germany,NaN,NaN,gerhard tremmel,gerhard,tremmel,g,germany,16.0,11.0,Germany,NaN,0.0,2017-07-17,NaN,NaN,NaN,NaN,NaN,gerhard tremmel,gerhard,tremmel,g,NaN,42.0,38.0,4.0,NaN,0.0,Swansea City,Retired,-,out,Summer,0.0,Premier League


In [129]:
# Record Linkage Step 1 - Create an indexer object
indexer = recordlinkage.Index()
indexer.block(left_on = ['first_initial_lower', 'birth_year', 'outfielder_goalkeeper', 'season'], 
              right_on = ['first_initial_lower', 'birth_year', 'outfielder_goalkeeper', 'season']
             )

<Index>

The next step is to build up all the potential candidates to check:

In [130]:
# Record Linkage Step 2 - Build up all the potential candidates to check:
candidates = indexer.index(df_fbref_merge, df_tm_merge_final)
print(len(candidates))

257763


Now that we have defined the left and right data sets and all the candidates, we can define how we want to perform the comparison logic using `Compare()`:

In [131]:
# Record Linkage Step 3 - Define how we to perform the comparison logic
compare = recordlinkage.Compare()

compare.string('first_name_lower',
               'first_name_lower',
               method='levenshtein',
               threshold=0.60,
               label='first_name'
              )

compare.string('last_name_lower',
               'last_name_lower',
               method='levenshtein',
               threshold=0.60,
               label='last_name'
              )

compare.string('country_lower',
               'country_lower',
               method='levenshtein',
               threshold=0.60,
               label='country'
              )

features = compare.compute(candidates, df_fbref_merge, df_tm_merge_final)

View the potential candidates

In [132]:
# Record Linkage Step 4 - view the potential candidates
features

first_name  last_name  country
240 28740         0.0        0.0      0.0
    28758         0.0        0.0      0.0
    29919         0.0        0.0      0.0
    29967         0.0        0.0      0.0
    30184         0.0        0.0      0.0
...               ...        ...      ...
193 1308          0.0        0.0      0.0
    5386          0.0        0.0      0.0
196 150           0.0        0.0      0.0
    1309          0.0        0.0      0.0
    5389          0.0        0.0      0.0

[257763 rows x 3 columns]

This DataFrame shows the results of all of the comparisons. There is one row for each row in the Company House and Fan360 DataFrames. The columns correspond to the comparisons we defined. A 1 is a match and 0 is not.

Given the large number of records with no matches, it is a little hard to see how many matches we might have. We can sum up the individual scores to see about the quality of the matches.

In [133]:
# Sum up the individual scores to see the quality of the matches.
features.sum(axis=1).value_counts().sort_index(ascending=False)

3.0      7341
2.0      7427
1.0     29502
0.0    213493
dtype: int64

To only include high-quality matches, let’s just take all the records with 3 matches out of 3 and create a total score column:

In [134]:
# Show records that have match by index number
potential_matches = features[features.sum(axis=1) >= 2].reset_index()
potential_matches

,level_0,level_1,first_name,last_name,country
0,240,32278,1.0,1.0,1.0
1,290,35813,1.0,1.0,1.0
2,540,32743,1.0,1.0,0.0
3,1048,31981,1.0,1.0,0.0
4,1089,34594,1.0,1.0,0.0
...,...,...,...,...,...
14763,15108,11482,1.0,1.0,0.0
14764,15109,11484,1.0,1.0,0.0
14765,15110,11485,1.0,1.0,0.0
14766,45,6333,1.0,1.0,1.0


In [135]:
potential_matches['Score'] = potential_matches.loc[:, 'first_name': 'country'].sum(axis=1)

# Display DataFrame of potential matches, potential_matches 
potential_matches

,level_0,level_1,first_name,last_name,country,Score
0,240,32278,1.0,1.0,1.0,3.0
1,290,35813,1.0,1.0,1.0,3.0
2,540,32743,1.0,1.0,0.0,2.0
3,1048,31981,1.0,1.0,0.0,2.0
4,1089,34594,1.0,1.0,0.0,2.0
...,...,...,...,...,...,...
14763,15108,11482,1.0,1.0,0.0,2.0
14764,15109,11484,1.0,1.0,0.0,2.0
14765,15110,11485,1.0,1.0,0.0,2.0
14766,45,6333,1.0,1.0,1.0,3.0


In [157]:
# Join Datasets

## Join Datasets


### Join the FBref Outfielder DataFrame to the potential matches DataFrame
df_merge_fbref_tm = pd.merge(potential_matches, df_fbref_merge, left_on='level_0', right_index=True, how='left')


### Join the TransferMarkt Outfielder DataFrame to the potential matches DataFrame
df_merge_fbref_tm = pd.merge(df_merge_fbref_tm, df_tm_merge_final, left_on='level_1', right_index=True, how='left')



## Data cleanup


### Rename columns - required otherwise 'birth_year' gets dropped
df_merge_fbref_tm = df_merge_fbref_tm.rename(columns={'birth_year_x': 'born'})


### Sort columns
df_merge_fbref_tm = df_merge_fbref_tm.sort_values(by=['season_x', 'player_name_fbref', 'tm_id_x', 'Score'], ascending=[True, True, True, False])


### Remove duplicates


#### Remove duplicate columns after join (contain '_y') and remove '_x' suffix from kept columns
df_merge_fbref_tm = df_merge_fbref_tm[df_merge_fbref_tm.columns.drop(list(df_merge_fbref_tm.filter(regex='_y')))]
df_merge_fbref_tm.columns = df_merge_fbref_tm.columns.str.replace('_x','')

#### Remove duplicate rows
df_merge_fbref_tm = df_merge_fbref_tm.drop_duplicates(subset=['player_name_fbref', 'season', 'Team Name', 'Comp'], keep='first')



### Rename columns
df_merge_fbref_tm = df_merge_fbref_tm.rename(columns={'born': 'birth_year',
                                                      'player_name': 'player_name_tm'
                                                     }
              
                                            )


### Sort columns
df_merge_fbref_tm = df_merge_fbref_tm.sort_values(by=['player_name_fbref', 'season'], ascending=[True, True])



## Determine columns to keep and remove

### Drop unnecessary columns
df_merge_fbref_tm = df_merge_fbref_tm.drop(['League Name', 'League ID', 'Score', 'level_0', 'level_1', 'first_name' , 'last_name', 'country'], axis=1)


### Define columns of interest

#### FBref players
lst_cols_fbref_players = ['player_name_fbref',
                          'fbref_id',
                          'url_fbref',
                          'first_initial_lower',
                          'first_name_lower',
                          'last_name_lower',
                          'birth_year',
                          'country_lower',
                          'outfielder_goalkeeper',
                          'season'
                         ]

#### TM Bio-Status, Valuations, and Transfers
lst_cols_tm = ['player_name_tm',
               'url_tm',
               'tm_id',
               'first_initial_lower',
               'first_name_lower',
               'last_name_lower',
               'birth_year',
               'country_lower',
               'outfielder_goalkeeper',
               'tm_id'
              ]


### Combine all columns of interest into a single list
lst_fbref_tm_select = list(lst_cols_fbref_players)
lst_fbref_tm_select.extend(x for x in lst_cols_tm if x not in lst_fbref_tm_select)


### Determine columns not of interest as separate list
lst_fbref_tm_non_select = list(set(list(df_merge_fbref_tm.columns)) - set(lst_fbref_tm_select))

### Define all columns
lst_fbref_tm_all = list(df_merge_fbref_tm.columns)


## Select columns of interest
df_merge_fbref_tm_select = df_merge_fbref_tm[lst_fbref_tm_select]

In [158]:
print('No. rows in FBref Players DataFrame before join to merged TM data: {}'.format(len(df_fbref_merge)))
print('No. rows in DataFrame AFTER join: {}\n'.format(len(df_merge_fbref_tm_select)))
print('-'*10+'\n')
print('Variance in rows before and after join: {}\n'.format(len(df_merge_fbref_tm_select) - len(df_fbref_merge)))

No. rows in FBref Players DataFrame before join to merged TM data: 13986
No. rows in DataFrame AFTER join: 11744

----------

Variance in rows before and after join: -2242



In [159]:
df_fbref_players.shape

(13680, 209)

In [160]:
# Display DataFrame
df_merge_fbref_tm.head()

,player_name_fbref,url_fbref,url_tm,tm_id,fbref_id,Nation,Pos,Squad,Comp,Age,birth_year,MP,Starts,Min,90s,Gls,Ast,G-PK,PK,PKatt,CrdY,CrdR,Gls.1,Ast.1,G+A,G-PK.1,G+A-PK,xG,npxG,xA,npxG+xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA.1,Matches,Sh,SoT,SoT%,Sh/90,SoT/90,G/Sh,G/SoT,Dist,FK,npxG/Sh,G-xG,np:G-xG,Cmp,Att,Cmp%,TotDist,PrgDist,Cmp.1,Att.1,Cmp%.1,Cmp.2,Att.2,Cmp%.2,Cmp.3,Att.3,Cmp%.3,A-xA,KP,1/3,PPA,CrsPA,Prog,Live,Dead,TB,Press,Sw,Crs,CK,In,Out,Str,Ground,Low,High,Left,Right,Head,TI,Other,Off,Out.1,Int,Blocks,SCA,SCA90,PassLive,PassDead,Drib,Fld,Def,GCA,GCA90,PassLive.1,PassDead.1,Drib.1,Sh.1,Fld.1,Def.1,Tkl,TklW,Def 3rd,Mid 3rd,Att 3rd,Tkl.1,Tkl%,Past,Succ,%,Def 3rd.1,Mid 3rd.1,Att 3rd.1,ShSv,Pass,Tkl+Int,Clr,Err,Touches,Def Pen,Att Pen,Succ%,#Pl,Megs,Carries,CPA,Mis,Dis,Targ,Rec,Rec%,Prog.1,Mn/MP,Min%,Mn/Start,Compl,Subs,Mn/Sub,unSub,PPM,onG,onGA,+/-,+/-90,On-Off,onxG,onxGA,xG+/-,xG+/-90,On-Off.1,2CrdY,Fls,PKwon,PKcon,OG,Recov,Won,Lost,Won%,season,Team Name,Team Country,Player Lower,First Name Lower,Last Name Lower,First Initial Lower,Team Country Lower,Nationality Code,Nationality Cleaned,Position Grouped,outfielder_goalkeeper,GA,GA90,SoTA,Saves,Save%,W,D,L,CS,CS%,PKA,PKsv,PKm,Save%.1,PSxG,PSxG/SoT,PSxG+/-,/90,Thr,Launch%,AvgLen,Launch%.1,AvgLen.1,Opp,Stp,Stp%,#OPA,#OPA/90,AvgDist,player_name_lower,first_name_lower,last_name_lower,first_initial_lower,country_lower,player_name_tm,club,current_club,league_code,current_age,market_value_gbp,market_value_eur,dob,pob,position,position_code,position_grouped,height,foot,citizenship,second_citizenship,player_agent,birth_day,birth_month,cob,current_club_country,market_value_euros,joined,contract_expires,contract_option,on_loan_from,on_loan_from_country,loan_contract_expiry,name_lower,firstname_lower,lastname_lower,firstinitial_lower,league_country_lower,age,age_when_joining,years_since_joining,years_until_contract_expiry,market_value_pounds,club_name,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name
0,Aaron Connolly,https://fbref.com/en/players/27c01749/Aaron-Co...,https://www.transfermarkt.com/aaron-connolly/p...,434207,27c01749,ie IRL,FW,Brighton,eng Premier League,19.0,2000.0,24.0,14.0,1258.0,14.0,3.0,1.0,3.0,0.0,0.0,0.0,0.0,0.21,0.07,0.29,0.21,0.29,3.2,3.2,0.3,3.5,0.23,0.02,0.25,0.23,0.25,Matches,38.0,13.0,34.2,2.72,0.93,0.08,0.23,15.9,0.0,0.08,-0.2,-0.2,126.0,163.0,77.3,1739.0,242.0,76.0,92.0,82.6,31.0,39.0,79.5,7.0,11.0,63.6,0.7,6.0,6.0,2.0,0.0,10.0,148.0,15.0,1.0,50.0,0.0,7.0,0.0,0.0,0.0,0.0,90.0,52.0,21.0,27.0,107.0,13.0,1.0,6.0,0.0,1.0,4.0,10.0,25.0,1.79,7.0,0.0,3.0,9.0,3.0,5.0,0.36,1.0,0.0,1.0,1.0,2.0,0.0,12.0,8.0,1.0,5.0,6.0,3.0,25.0,9.0,69.0,29.5,14.0,94.0,126.0,0.0,7.0,17.0,1.0,0.0,349.0,2.0,61.0,37.5,6.0,1.0,228.0,12.0,42.0,34.0,535.0,235.0,43.9,99.0,52.0,36.8,72.0,0.0,10.0,26.0,4.0,1.13,18.0,22.0,-4.0,-0.29,0.17,15.6,19.8,-4.2,-0.30,0.08,0.0,16.0,2.0,0.0,0.0,54.0,14.0,48.0,22.6,2019/2020,Brighton,England,aaron connolly,aaron,connolly,a,england,IRL,Ireland,Forward,Outfielder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,aaron connolly,aaron,connolly,a,ireland,aaron connolly,Brighton Hove Albion,brighton & hove albion,GB1,21.0,4050000.0,4500000,2000-01-28,Galway,attack - Centre-Forward,ST,Forward,175.0,right,Ireland,NaN,PLG,28.0,1.0,Ireland,england,7000000.0,2019-07-01,2024-06-30,NaN,NaN,NaN,NaN,aaron connolly,aaron,connolly,a,england,21.0,19.0,2.0,2.0,6300000.0,Brighton & Hove Albion,Brighton U23,-,in,Summer,0.0,Premier League
16,Aaron Connolly,https://fbref.com/en/players/27c01749/Aaron-Co...,https://www.transfermarkt.com/aaron-connolly/p...,434207,27c01749,ie IRL,FW,Brighton,eng Premier League,20.0,2000.0,17.0,9.0,791.0,8.8,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.23,0.11,0.34,0.23,0.34,3.5,3.5,0.2,3.7,0.40,0.02,0.42,0.40,0.42,Matches,23.0,8.0,34.8,2.62,0.91,0.09,0.25,13.7,0.0,0.15,-1.5,-1.5,79.0,101.0,78.2,1147.0,165.0,45.0,56.0,80.4,26.0,30.0,86.7,4.0,5.0,80.0,0.8,5.0,2.0,1.0,0.0,3.0,

In [161]:
# Display DataFrame
df_merge_fbref_tm_select

,player_name_fbref,fbref_id,url_fbref,first_initial_lower,first_name_lower,last_name_lower,birth_year,country_lower,outfielder_goalkeeper,season,player_name_tm,url_tm,tm_id
0,Aaron Connolly,27c01749,https://fbref.com/en/players/27c01749/Aaron-Co...,a,aaron,connolly,2000.0,ireland,Outfielder,2019/2020,aaron connolly,https://www.transfermarkt.com/aaron-connolly/p...,434207
16,Aaron Connolly,27c01749,https://fbref.com/en/players/27c01749/Aaron-Co...,a,aaron,connolly,2000.0,ireland,Outfielder,2020/2021,aaron connolly,https://www.transfermarkt.com/aaron-connolly/p...,434207
32,Aaron Connolly,27c01749,https://fbref.com/en/players/27c01749/Aaron-Co...,a,aaron,connolly,2000.0,ireland,Outfielder,2021/2022,aaron connolly,https://www.transfermarkt.com/aaron-connolly/p...,434207
44,Aaron Cresswell,4f974391,https://fbref.com/en/players/4f974391/Aaron-Cr...,a,aaron,cresswell,1989.0,england,Outfielder,2017/2018,aaron cresswell,https://www.transfermarkt.com/aaron-cresswell/...,92571
72,Aaron Cresswell,4f974391,https://fbref.com/en/players/4f974391/Aaron-Cr...,a,aaron,cresswell,1989.0,england,Outfielder,2018/2019,aaron cresswell,https://www.transfermarkt.com/aaron-cresswell/...,92571
...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,Ãngel RodrÃ­guez,8cfc2f69,https://fbref.com/en/players/8cfc2f69/Angel-Ro...,a,angel,rodraguez,1987.0,spain,Outfielder,2021/2022,ángel rodríguez,https://www.transfermarkt.com/angel-rodriguez/...,51510
449,Ãscar Gil,6652cf96,https://fbref.com/en/players/6652cf96/Oscar-Gil,a,ascar,gil,1997.0,spain,Outfielder,2021/2022,asier villalibre,https://www.transfermarkt.com/oscar-gil/profil...,544106
474,Ãscar Melendo,379652be,https://fbref.com/en/players/379652be/Oscar-Me...,a,ascar,melendo,1997.0,spain,Outfielder,2017/2018,asier villalibre,https://www.transfermarkt.com/oscar-melendo/pr...,448344
295,Ãscar Melendo,379652be,https://fbref.com/en/players/379652be/Oscar-Me...,a,ascar,melendo,1997.0,spain,Outfielder,2018/2019,asier villalibre,https://www.transfermarkt.com/oscar-melendo/pr...,448344


In [150]:
lst_fbref_tm_select

['player_name_fbref',
 'fbref_id',
 'first_initial_lower',
 'first_name_lower',
 'last_name_lower',
 'birth_year',
 'country_lower',
 'outfielder_goalkeeper',
 'season',
 'player_name_tm',
 'url_tm',
 'tm_id']

In [151]:
lst_fbref_tm_non_select

['Att 3rd',
 'PK',
 '#OPA',
 'xA.1',
 'Int',
 'Megs',
 'Cmp%.1',
 'Drib.1',
 'on_loan_from',
 'Tkl.1',
 'Fls',
 'Launch%',
 'Crs',
 'Att',
 'Att.2',
 'AvgLen',
 'Save%.1',
 'In',
 'lastname_lower',
 'firstinitial_lower',
 'L',
 'ShSv',
 'Def 3rd.1',
 'years_since_joining',
 'onGA',
 'onxGA',
 'Def Pen',
 'Last Name Lower',
 'Dead',
 'Gls',
 'Drib',
 'Off',
 'PrgDist',
 'Prog',
 'onG',
 'birth_month',
 'Press',
 'AvgDist',
 'SCA',
 'Opp',
 '#OPA/90',
 'A-xA',
 'TotDist',
 'fee_cleaned',
 'SoTA',
 'Touches',
 'transfer_period',
 'Ast.1',
 'Low',
 'CrdY',
 'First Name Lower',
 'PKwon',
 'GCA90',
 'dob',
 'FK',
 'Position Grouped',
 'second_citizenship',
 'current_club',
 'Matches',
 'firstname_lower',
 'Tkl%',
 'G-PK.1',
 'Att Pen',
 'High',
 'Head',
 'Tkl',
 'Dis',
 'G-xG',
 'Targ',
 'pob',
 'Cmp.2',
 'club_involved_name',
 'Def.1',
 '#Pl',
 'Att 3rd.1',
 'G+A-PK',
 'Stp',
 'xG+/-90',
 'unSub',
 'PSxG+/-',
 'Sh/90',
 'market_value_pounds',
 'transfer_movement',
 'name_lower',
 'Subs',
 '

<a id='section5.3.2'></a>

#### <a id='#section5.3.2'>5.3.2. Join the Capology dataset</a>

In [162]:
df_capology.head()

,player,season,league,team,position,outfielder_goalkeeper,age,country,weekly_gross_base_salary_gbp,annual_gross_base_salary_gbp,adj_current_gross_base_salary_gbp,estimated_gross_total_gbp,current_contract_status,current_contract_expiration,current_contract_length,player_name_lower,first_name_lower,last_name_lower,first_initial_lower,country_lower
645,Andreas Luthe,2017/2018,Bundesliga,Augsburg,Goalkeeper,Goalkeeper,30,Germany,6147.0,319688.0,325953.0,NaN,NaN,NaN,NaN,andreas luthe,andreas,luthe,a,germany
647,Christoph Janker,2017/2018,Bundesliga,Augsburg,Defender,Outfielder,32,Germany,7585.0,394432.0,402161.0,NaN,NaN,NaN,NaN,christoph janker,christoph,janker,c,germany
648,Daniel Baier,2017/2018,Bundesliga,Augsburg,Midfielder,Outfielder,33,Germany,34185.0,1777646.0,1812482.0,NaN,NaN,NaN,NaN,daniel baier,daniel,baier,d,germany
651,Efkan Bekiroglu,2017/2018,Bundesliga,Augsburg,Midfielder,Outfielder,21,Germany,761.0,39623.0,40399.0,NaN,NaN,NaN,NaN,efkan bekiroglu,efkan,bekiroglu,e,germany
652,Erik Thommy,2017/2018,Bundesliga,Augsburg,Midfielder,Outfielder,22,Germany,7585.0,394432.0,402161.0,NaN,NaN,NaN,NaN,erik thommy,erik,thommy,e,germany


In [163]:
# Record Linkage Step 1 - Create an indexer object
indexer = recordlinkage.Index()
indexer.block(left_on = ['first_initial_lower', 'outfielder_goalkeeper', 'season'], 
              right_on = ['first_initial_lower', 'outfielder_goalkeeper', 'season']
             )

<Index>

In [164]:
# Record Linkage Step 2 - Build up all the potential candidates to check:
candidates = indexer.index(df_merge_fbref_tm, df_capology)
print(len(candidates))

857503


In [165]:
# Record Linkage Step 3 - Define how we to perform the comparison logic
compare = recordlinkage.Compare()

compare.string('first_name_lower',
               'first_name_lower',
               method='levenshtein',
               threshold=0.60,
               label='first_name'
              )

compare.string('last_name_lower',
               'last_name_lower',
               method='levenshtein',
               threshold=0.60,
               label='last_name'
              )

compare.string('country_lower',
               'country_lower',
               method='levenshtein',
               threshold=0.60,
               label='country'
              )

features = compare.compute(candidates, df_merge_fbref_tm, df_capology)

In [166]:
# Record Linkage Step 4 - view the potential candidates
features

first_name  last_name  country
0     1878          0.0        0.0      0.0
      1964          0.0        0.0      0.0
      1997          0.0        0.0      0.0
      1998          0.0        0.0      0.0
      2032          0.0        0.0      0.0
...                 ...        ...      ...
14747 15803         0.0        0.0      0.0
14754 15803         0.0        0.0      0.0
14757 15803         0.0        0.0      0.0
14759 15803         0.0        0.0      0.0
14762 15803         0.0        0.0      0.0

[857503 rows x 3 columns]

In [167]:
# Sum up the individual scores to see the quality of the matches.
features.sum(axis=1).value_counts().sort_index(ascending=False)

3.0      4842
2.0     20569
1.0    147463
0.0    684629
dtype: int64

In [168]:
# Show records that have match by index number
potential_matches = features[features.sum(axis=1) >= 2].reset_index()
potential_matches

,level_0,level_1,first_name,last_name,country
0,96,14909,1.0,0.0,1.0
1,96,15090,1.0,0.0,1.0
2,96,15294,1.0,1.0,1.0
3,200,14909,1.0,1.0,1.0
4,200,15090,1.0,0.0,1.0
...,...,...,...,...,...
25406,14744,6473,1.0,1.0,1.0
25407,14744,17924,1.0,1.0,1.0
25408,14745,6620,1.0,1.0,1.0
25409,14746,7787,1.0,1.0,1.0


In [169]:
potential_matches['Score'] = potential_matches.loc[:, 'first_name': 'country'].sum(axis=1)

# Display DataFrame of potential matches, potential_matches 
potential_matches

,level_0,level_1,first_name,last_name,country,Score
0,96,14909,1.0,0.0,1.0,2.0
1,96,15090,1.0,0.0,1.0,2.0
2,96,15294,1.0,1.0,1.0,3.0
3,200,14909,1.0,1.0,1.0,3.0
4,200,15090,1.0,0.0,1.0,2.0
...,...,...,...,...,...,...
25406,14744,6473,1.0,1.0,1.0,3.0
25407,14744,17924,1.0,1.0,1.0,3.0
25408,14745,6620,1.0,1.0,1.0,3.0
25409,14746,7787,1.0,1.0,1.0,3.0


In [170]:
df_merge_fbref_tm[df_merge_fbref_tm.index == 2929]

,player_name_fbref,url_fbref,url_tm,tm_id,fbref_id,Nation,Pos,Squad,Comp,Age,birth_year,MP,Starts,Min,90s,Gls,Ast,G-PK,PK,PKatt,CrdY,CrdR,Gls.1,Ast.1,G+A,G-PK.1,G+A-PK,xG,npxG,xA,npxG+xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA.1,Matches,Sh,SoT,SoT%,Sh/90,SoT/90,G/Sh,G/SoT,Dist,FK,npxG/Sh,G-xG,np:G-xG,Cmp,Att,Cmp%,TotDist,PrgDist,Cmp.1,Att.1,Cmp%.1,Cmp.2,Att.2,Cmp%.2,Cmp.3,Att.3,Cmp%.3,A-xA,KP,1/3,PPA,CrsPA,Prog,Live,Dead,TB,Press,Sw,Crs,CK,In,Out,Str,Ground,Low,High,Left,Right,Head,TI,Other,Off,Out.1,Int,Blocks,SCA,SCA90,PassLive,PassDead,Drib,Fld,Def,GCA,GCA90,PassLive.1,PassDead.1,Drib.1,Sh.1,Fld.1,Def.1,Tkl,TklW,Def 3rd,Mid 3rd,Att 3rd,Tkl.1,Tkl%,Past,Succ,%,Def 3rd.1,Mid 3rd.1,Att 3rd.1,ShSv,Pass,Tkl+Int,Clr,Err,Touches,Def Pen,Att Pen,Succ%,#Pl,Megs,Carries,CPA,Mis,Dis,Targ,Rec,Rec%,Prog.1,Mn/MP,Min%,Mn/Start,Compl,Subs,Mn/Sub,unSub,PPM,onG,onGA,+/-,+/-90,On-Off,onxG,onxGA,xG+/-,xG+/-90,On-Off.1,2CrdY,Fls,PKwon,PKcon,OG,Recov,Won,Lost,Won%,season,Team Name,Team Country,Player Lower,First Name Lower,Last Name Lower,First Initial Lower,Team Country Lower,Nationality Code,Nationality Cleaned,Position Grouped,outfielder_goalkeeper,GA,GA90,SoTA,Saves,Save%,W,D,L,CS,CS%,PKA,PKsv,PKm,Save%.1,PSxG,PSxG/SoT,PSxG+/-,/90,Thr,Launch%,AvgLen,Launch%.1,AvgLen.1,Opp,Stp,Stp%,#OPA,#OPA/90,AvgDist,player_name_lower,first_name_lower,last_name_lower,first_initial_lower,country_lower,player_name_tm,club,current_club,league_code,current_age,market_value_gbp,market_value_eur,dob,pob,position,position_code,position_grouped,height,foot,citizenship,second_citizenship,player_agent,birth_day,birth_month,cob,current_club_country,market_value_euros,joined,contract_expires,contract_option,on_loan_from,on_loan_from_country,loan_contract_expiry,name_lower,firstname_lower,lastname_lower,firstinitial_lower,league_country_lower,age,age_when_joining,years_since_joining,years_until_contract_expiry,market_value_pounds,club_name,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name


In [171]:
df_capology[df_capology.index == 6473]

,player,season,league,team,position,outfielder_goalkeeper,age,country,weekly_gross_base_salary_gbp,annual_gross_base_salary_gbp,adj_current_gross_base_salary_gbp,estimated_gross_total_gbp,current_contract_status,current_contract_expiration,current_contract_length,player_name_lower,first_name_lower,last_name_lower,first_initial_lower,country_lower
6473,Zinédine Machach,2017/2018,Ligue 1,Toulouse,Midfielder,Outfielder,21,France,0.0,0.0,0.0,NaN,NaN,NaN,NaN,zinedine machach,zinedine,machach,z,france


In [186]:
# Join Datasets

## Join Datasets


### Join the FBref Outfielder DataFrame to the potential matches DataFrame
df_merge_fbref_tm_capology = pd.merge(potential_matches, df_merge_fbref_tm, left_on='level_0', right_index=True, how='left')


### Join the TransferMarkt Outfielder DataFrame to the potential matches DataFrame
df_merge_fbref_tm_capology = pd.merge(df_merge_fbref_tm_capology, df_capology, left_on='level_1', right_index=True, how='left')



## Data cleanup


### Rename columns - required otherwise 'birth_year' gets dropped
df_merge_fbref_tm_capology = df_merge_fbref_tm_capology.rename(columns={'birth_year_x': 'born'})


### Sort columns
df_merge_fbref_tm_capology = df_merge_fbref_tm_capology.sort_values(by=['season_x', 'player_name_fbref', 'tm_id', 'Score'], ascending=[True, True, True, False])


### Remove duplicates


#### Remove duplicate columns after join (contain '_y') and remove '_x' suffix from kept columns
df_merge_fbref_tm_capology = df_merge_fbref_tm_capology[df_merge_fbref_tm_capology.columns.drop(list(df_merge_fbref_tm_capology.filter(regex='_y')))]
df_merge_fbref_tm_capology.columns = df_merge_fbref_tm_capology.columns.str.replace('_x', '')

#### Remove duplicate rows
df_merge_fbref_tm_capology = df_merge_fbref_tm_capology.drop_duplicates(subset=['player_name_fbref', 'tm_id', 'fbref_id', 'season', 'Team Name', 'Comp'], keep='first')



### Rename columns
df_merge_fbref_tm_capology = df_merge_fbref_tm_capology.rename(columns={'born': 'birth_year',
                                                                        'player_name': 'player_name_tm'
                                                                       }

                                                              )


### Sort columns
df_merge_fbref_tm_capology = df_merge_fbref_tm_capology.sort_values(by=['player_name_fbref', 'season'], ascending=[True, True])



## Determine columns to keep and remove


### Drop unnecessary columns
df_merge_fbref_tm_capology = df_merge_fbref_tm_capology.drop(['Score', 'level_0', 'level_1', 'first_name' , 'last_name', 'country'], axis=1)


### Define columns of interest

#### Capology
lst_cols_capology = ['player',
                     'first_initial_lower',
                     'first_name_lower',
                     'last_name_lower',
                     'country_lower',
                     'outfielder_goalkeeper',
                     'season'
                    ]


### Combine all columns of interest into a single list
lst_fbref_tm_capology_select = list(lst_cols_capology)
lst_fbref_tm_capology_select.extend(x for x in lst_fbref_tm_select if x not in lst_fbref_tm_capology_select)


### Determine columns not of interest as separate list
lst_fbref_tm_capology_select = list(set(list(df_merge_fbref_tm_capology.columns)) - set(lst_fbref_tm_capology_select))


### Define all columns
lst_fbref_tm_capology_all = list(df_merge_fbref_tm_capology.columns)



## Select columns of interest
df_merge_fbref_tm_capology_select = df_merge_fbref_tm_capology[lst_fbref_tm_capology_select]

In [187]:
print('No. rows in FBref_TM DataFrame before join to Capology data: {}'.format(len(df_merge_fbref_tm_select)))
print('No. rows in DataFrame AFTER join: {}\n'.format(len(df_merge_fbref_tm_capology_select)))
print('-'*10+'\n')
print('Variance in rows before and after join: {}\n'.format(len(df_merge_fbref_tm_capology_select) - len(df_merge_fbref_tm_select)))

No. rows in FBref_TM DataFrame before join to Capology data: 11744
No. rows in DataFrame AFTER join: 4915

----------

Variance in rows before and after join: -6829



In [181]:
# Display DataFrame
df_merge_fbref_tm_capology.head()

,player_name_fbref,url_fbref,url_tm,tm_id,fbref_id,Nation,Pos,Squad,Comp,Age,MP,Starts,Min,90s,Gls,Ast,G-PK,PK,PKatt,CrdY,CrdR,Gls.1,Ast.1,G+A,G-PK.1,G+A-PK,xG,npxG,xA,npxG+xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA.1,Matches,Sh,SoT,SoT%,Sh/90,SoT/90,G/Sh,G/SoT,Dist,FK,npxG/Sh,G-xG,np:G-xG,Cmp,Att,Cmp%,TotDist,PrgDist,Cmp.1,Att.1,Cmp%.1,Cmp.2,Att.2,Cmp%.2,Cmp.3,Att.3,Cmp%.3,A-xA,KP,1/3,PPA,CrsPA,Prog,Live,Dead,TB,Press,Sw,Crs,CK,In,Out,Str,Ground,Low,High,Left,Right,Head,TI,Other,Off,Out.1,Int,Blocks,SCA,SCA90,PassLive,PassDead,Drib,Fld,Def,GCA,GCA90,PassLive.1,PassDead.1,Drib.1,Sh.1,Fld.1,Def.1,Tkl,TklW,Def 3rd,Mid 3rd,Att 3rd,Tkl.1,Tkl%,Past,Succ,%,Def 3rd.1,Mid 3rd.1,Att 3rd.1,ShSv,Pass,Tkl+Int,Clr,Err,Touches,Def Pen,Att Pen,Succ%,#Pl,Megs,Carries,CPA,Mis,Dis,Targ,Rec,Rec%,Prog.1,Mn/MP,Min%,Mn/Start,Compl,Subs,Mn/Sub,unSub,PPM,onG,onGA,+/-,+/-90,On-Off,onxG,onxGA,xG+/-,xG+/-90,On-Off.1,2CrdY,Fls,PKwon,PKcon,OG,Recov,Won,Lost,Won%,season,Team Name,Team Country,Player Lower,First Name Lower,Last Name Lower,First Initial Lower,Team Country Lower,Nationality Code,Nationality Cleaned,Position Grouped,outfielder_goalkeeper,GA,GA90,SoTA,Saves,Save%,W,D,L,CS,CS%,PKA,PKsv,PKm,Save%.1,PSxG,PSxG/SoT,PSxG+/-,/90,Thr,Launch%,AvgLen,Launch%.1,AvgLen.1,Opp,Stp,Stp%,#OPA,#OPA/90,AvgDist,player_name_lower,first_name_lower,last_name_lower,first_initial_lower,country_lower,player_name_tm,club,current_club,league_code,current_age,market_value_gbp,market_value_eur,dob,pob,position,position_code,position_grouped,height,foot,citizenship,second_citizenship,player_agent,birth_day,birth_month,cob,current_club_country,market_value_euros,joined,contract_expires,contract_option,on_loan_from,on_loan_from_country,loan_contract_expiry,name_lower,firstname_lower,lastname_lower,firstinitial_lower,league_country_lower,age,age_when_joining,years_since_joining,years_until_contract_expiry,market_value_pounds,club_name,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,player,league,team,weekly_gross_base_salary_gbp,annual_gross_base_salary_gbp,adj_current_gross_base_salary_gbp,estimated_gross_total_gbp,current_contract_status,current_contract_expiration,current_contract_length
1260,Aaron Cresswell,https://fbref.com/en/players/4f974391/Aaron-Cr...,https://www.transfermarkt.com/aaron-cresswell/...,92571,4f974391,eng ENG,DF,West Ham,eng Premier League,27.0,36.0,35.0,3069.0,34.1,1.0,3.0,1.0,0.0,0.0,7.0,0.0,0.03,0.09,0.12,0.03,0.12,0.8,0.8,2.8,3.6,0.02,0.08,0.10,0.02,0.10,Matches,21.0,6.0,28.6,0.62,0.18,0.05,0.17,28.1,8.0,0.04,0.2,0.2,1224.0,1708.0,71.7,23519.0,10212.0,560.0,623.0,89.9,472.0,587.0,80.4,183.0,449.0,40.8,0.2,35.0,117.0,21.0,14.0,96.0,1343.0,365.0,1.0,222.0,83.0,93.0,67.0,35.0,15.0,9.0,893.0,293.0,522.0,1329.0,78.0,59.0,210.0,5.0,15.0,44.0,39.0,52.0,62.0,1.82,35.0,21.0,1.0,3.0,0.0,9.0,0.26,6.0,3.0,0.0,0.0,0.0,0.0,38.0,18.0,15.0,18.0,5.0,17.0,53.1,15.0,115.0,32.1,181.0,123.0,54.0,0.0,38.0,90.0,133.0,0.0,2050.0,125.0,17.0,33.3,7.0,0.0,1071.0,2.0,18.0,19.0,1171.0,1094.0,93.4,31.0,85.0,89.7,NaN,30.0,1.0,NaN,1.0,1.14,45.0,60.0,-15.0,-0.44,0.84,38.0,51.5,-13.5,-0.40,1.09,0.0,20.0,0.0,0.0,0.0,277.0,70.0,57.0,55.1,2017/2018,West Ham,England,aaron cresswell,aaron,cresswell,a,england,ENG,England,Defender,Outfielder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,aaron cresswell,aaron,cresswell,a,england,aaron cresswell,West Ham United,west ham united,GB1,31.0,10800000.0,12000000,1989-12-15,Liverpool,Defender - Left-Back,LB,Defender,170.0,left,England,NaN,Unique Sports Management,15.0,12.0,England,england,5000000.0,2014-07-03,2023-06-30,NaN,NaN,NaN,NaN,aaron cresswell,aaron,cresswell,a,england,31.0,24.0,7.0,1.0,4500000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aaron Cresswell,Premier League,West Ham,50000.0,2600000.0,2671365.0,NaN,NaN,NaN,NaN
2261,Aaron Cresswell,https://fbref.com/en/players/4f974391/Aaron-Cr...,https://www.transfermarkt.com/aaron-cresswell/...,92571,4f974391,eng ENG,DF

In [185]:
# Display DataFrame
df_merge_fbref_tm_capology_select.head()

,Att 3rd,PK,#OPA,xA.1,Int,Megs,Cmp%.1,Drib.1,on_loan_from,Tkl.1,Fls,Launch%,Crs,Att,Att.2,AvgLen,Save%.1,In,lastname_lower,firstinitial_lower,L,ShSv,Def 3rd.1,years_since_joining,onGA,onxGA,Def Pen,Last Name Lower,Dead,Gls,Drib,Off,PrgDist,Prog,onG,birth_month,Press,team,AvgDist,SCA,Opp,#OPA/90,A-xA,TotDist,fee_cleaned,SoTA,Touches,transfer_period,Ast.1,Low,CrdY,First Name Lower,PKwon,GCA90,dob,FK,Position Grouped,second_citizenship,current_club,Matches,firstname_lower,Tkl%,G-PK.1,Att Pen,High,Head,Tkl,Dis,G-xG,Targ,pob,Cmp.2,club_involved_name,Def.1,#Pl,Att 3rd.1,current_contract_length,G+A-PK,Stp,xG+/-90,unSub,PSxG+/-,Sh/90,market_value_pounds,transfer_movement,name_lower,Subs,league_country_lower,fee,citizenship,PSxG/SoT,KP,xG+xA,Mis,Right,club,Player Lower,player_agent,npxG/Sh,market_value_gbp,SoT%,current_age,Ground,age_when_joining,TB,G/Sh,SoT,league,Blocks,cob,annual_gross_base_salary_gbp,years_until_contract_expiry,npxG.1,Saves,GA90,foot,height,npxG,First Initial Lower,Squad,Past,Sw,CK,G-PK,Err,PSxG,PKm,MP,TI,loan_contract_expiry,Ast,Rec,position_code,contract_option,xG,xG+/-,Clr,2CrdY,Comp,Nation,Cmp,On-Off.1,Left,GCA,weekly_gross_base_salary_gbp,np:G-xG,On-Off,current_club_country,on_loan_from_country,Mn/Start,PPA,Pass,W,Cmp.3,xA,Sh,CPA,PassDead.1,SoT/90,Prog.1,PassDead,onxG,player_name_lower,Lost,Won%,npxG+xA.1,age,Sh.1,Dist,Att.1,Fld.1,joined,current_contract_status,current_contract_expiration,Thr,Min%,Mid 3rd.1,SCA90,Tkl+Int,Team Name,CrsPA,CS,%,Att.3,Mn/Sub,Recov,position_grouped,G+A,Nationality Cleaned,club_name,PKsv,PKA,Rec%,/90,contract_expires,Team Country,+/-,Out.1,GA,estimated_gross_total_gbp,Def,Live,Save%,Starts,Min,position,market_value_eur,PassLive,league_code,Won,Carries,TklW,league_name,Def 3rd,PassLive.1,OG,Mid 3rd,market_value_euros,Age,adj_current_gross_base_salary_gbp,birth_day,Cmp%.3,+/-90,Stp%,Nationality Code,Succ%,D,xG.1,Cmp.1,Cmp%.2,PKatt,CS%,1/3,Compl,Gls.1,npxG+xA,G/SoT,CrdR,Fld,Launch%.1,Cmp%,Out,90s,PKcon,Succ,AvgLen.1,PPM,Pos,Mn/MP,Str,Team Country Lower,Other
1260,5.0,0.0,NaN,0.08,39.0,0.0,89.9,0.0,NaN,17.0,20.0,NaN,93.0,1708.0,587.0,NaN,NaN,35.0,cresswell,a,NaN,0.0,181.0,7.0,60.0,51.5,125.0,cresswell,365.0,1.0,1.0,15.0,10212.0,96.0,45.0,12.0,222.0,West Ham,NaN,62.0,NaN,NaN,0.2,23519.0,NaN,NaN,2050.0,NaN,0.09,293.0,7.0,aaron,0.0,0.26,1989-12-15,8.0,Defender,NaN,west ham united,Matches,aaron,53.1,0.03,17.0,522.0,59.0,38.0,19.0,0.2,1171.0,Liverpool,472.0,NaN,0.0,7.0,54.0,NaN,0.12,NaN,-0.40,1.0,NaN,0.62,4500000.0,NaN,aaron cresswell,1.0,england,NaN,England,NaN,35.0,0.10,18.0,78.0,West Ham United,aaron cresswell,Unique Sports Management,0.04,10800000.0,28.6,31.0,893.0,24.0,1.0,0.05,6.0,Premier League,52.0,England,2600000.0,1.0,0.02,NaN,NaN,left,170.0,0.8,a,West Ham,15.0,83.0,67.0,1.0,0.0,NaN,NaN,36.0,210.0,NaN,3.0,1094.0,LB,NaN,0.8,-13.5,133.0,0.0,eng Premier League,eng ENG,1224.0,1.09,1329.0,9.0,50000.0,0.2,0.84,england,NaN,NaN,21.0,38.0,NaN,183.0,2.8,21.0,2.0,3.0,0.18,31.0,21.0,38.0,aaron cresswell,57.0,55.1,0.10,31.0,0.0,28.1,623.0,0.0,2014-07-03,NaN,NaN,NaN,89.7,123.0,1.82,90.0,West Ham,14.0,NaN,32.1,449.0,NaN,277.0,Defender,0.12,England,NaN,NaN,NaN,93.4,NaN,2023-06-30,England,-15.0,44.0,NaN,NaN,0.0,1343.0,NaN,35.0,3069.0,Defender - Left-Back,12000000,35.0,GB1,70.0,1071.0,18.0,NaN,15.0,6.0,0.0,18.0,5000000.0,27.0,2671365.0,15.0,40.8,-0.44,NaN,ENG,33.3,NaN,0.02,560.0,80.4,0.0,NaN,117.0,30.0,0.03,3.6,0.17,0.0,3.0,NaN,71.7,15.0,34.1,0.0,115.0,NaN,1.14,DF,85.0,9.0,england,5.0
2261,4.0,0.0,NaN,0.05,27.0,1.0,90.4,0.0,NaN,12.0,2.0,NaN,46.0,1070.0,371.0,NaN,NaN,0.0,cresswell,a,NaN,0.0,129.0,7.0,26.0,25.3,78.0,cresswell,216.0,0.0,0.0,2.0,5572.0,65.0,21.0,12.0,168.0,West Ham,NaN,29.0,NaN,NaN,0.1,13627.0,NaN,NaN,1266.0,NaN,0.06,235.0,1.0,aaron,0.0,0.11,1989-12-15,2.0,Defender,NaN,west ham united,Matches,aaron,42.9,0.00,36.0,193.0,27.0,30.0,13.0,-0.5,797.0,Liverpool,307.0,NaN,0.0,7.0,28.0,NaN,0.06,NaN,-0.30,7.0,NaN,0.62,4500000.0,NaN,aaron cresswell,2.0,england,NaN,England,NaN,16.0,0.08,11.0,51.0,W

## FROM HERE, CODE IS OLD

---

<a id='section6'></a>

## <a id='#section6'>6. ...
To create 'Golden ID' table.

In [ ]:
# Join the 
df_golden_id = pd.merge(df_fbref_players_select, df_merge_fbref_tm_select['player_name_fbref'], left_on='player', right_on='player_name_fbref', how='left')

# Join the 
df_golden_id = pd.merge(df_golden_id, df_merge_fbref_tm_select[['player_name_fbref', 'player_name_tm', 'id_tm']], left_on='player', right_on='player_name_fbref', how='left')

# Drop duplicates at each stage before joining a new dataset
df_golden_id  = df_golden_id.sort_values(by=['player'], ascending=[True]).drop_duplicates(subset=['player', 'nationality_cleaned', 'birth_year', 'outfielder_goalkeeper'], keep='first')

# Join the 
df_golden_id = pd.merge(df_golden_id, df_merge_fbref_capology_select['player_name_capology'], left_on='player', right_on='player_name_capology', how='left')

# Drop duplicates at each stage before joining a new dataset
df_golden_id  = df_golden_id.sort_values(by=['player'], ascending=[True]).drop_duplicates(subset=['player', 'nationality_cleaned', 'birth_year', 'outfielder_goalkeeper'], keep='first')

# Select columns of interest
cols_golden_id = ['player',
                  'player_name_tm',
                  'id_tm',
                  'player_name_capology'
                 ]

# Create more concise DataFrame using only columns of interest
df_golden_id_select = df_golden_id[cols_golden_id]

# Sort columns
df_golden_id_select = df_golden_id_select.sort_values(by=['player'], ascending=[True])

# Drop duplicates
df_golden_id_select = df_golden_id_select.drop_duplicates()

# Rename columns
df_golden_id_select = df_golden_id_select.rename(columns={'player': 'player_name_fbref'
                                                         }
                                                )

# Display DataFrame
df_golden_id_select.head()

In [ ]:
df_golden_id_select.shape

In [ ]:
df_fbref_players_select.shape

In [ ]:
# sort some of the duplication problems
# create a 'spot checks' section

---

<a id='section6'></a>

## <a id='#section6'>6. Final Player Performance and Valuation Dataset</a>
Using the 'Golden ID' table, df_golden_id, create a unified dataset of player information, including FBref player performance data, TransferMarkt bio and status (estimated valuation) as well as TransferMarkt recorded transfer data, with Capology player salaries.

This final dataset can be subsequently used for modeling and as dataset for a player valuation model.

In [ ]:
# Temporary FBref season fix

## Map Positions

### 
dict_season_short_grouped = {'17/18': '2017/2018',
                             '18/19': '2018/2019',
                             '19/20': '2019/2020',
                             '20/21': '2020/2021',
                             '21/22': '2021/2022',
                            }

### Map grouped positions to DataFrame
df_fbref_players['Season'] = df_fbref_players['Season'].map(dict_season_short_grouped)

In [ ]:
# Temporary Capology season fix

## Map Positions

### 
dict_season_full_grouped = {'2010-2011': '2010/2011',
                            '2011-2012': '2011/2012',
                            '2012-2013': '20122013',
                            '2013-2014': '2013/2014',
                            '2014-2015': '2014/2015',
                            '2015-2016': '2015/2016',
                            '2016-2017': '2016/2017',
                            '2017-2018': '2017/2018',
                            '2018-2019': '2018/2019',
                            '2019-2020': '2019/2020',
                            '2020-2021': '2020/2021',
                            '2021-2020': '2021/2022'
                           }

### Map grouped positions to DataFrame
df_capology['season'] = df_capology['season'].map(dict_season_full_grouped)

In [ ]:
# Join the 
df_final = pd.merge(df_fbref_players, df_golden_id_select, left_on='player', right_on='player_name_fbref', how='left')

# Join the 
df_final = pd.merge(df_final, df_tm_valuations, left_on=['player_name_tm', 'Season'], right_on=['player_name', 'season'], how='left')

# Drop duplicates at each stage before joining a new dataset
df_final = df_final.sort_values(by=['player', 'Season'], ascending=[True, True]).drop_duplicates(subset=['player', 'nationality_cleaned', 'birth_year_x', 'outfielder_goalkeeper_x', 'squad', 'Season'], keep='first')

# Join the 
df_final = pd.merge(df_final, df_capology, left_on=['player_name_capology', 'Season'], right_on=['player', 'season'], how='left')

# Drop duplicates at each stage before joining a new dataset
df_final = df_final.sort_values(by=['player_x', 'Season'], ascending=[True, True]).drop_duplicates(subset=['player_x', 'nationality_cleaned', 'birth_year_x', 'outfielder_goalkeeper_x', 'squad', 'Season'], keep='first')

# Rename columns

In [ ]:
df_final

In [ ]:
df_fbref_players.shape

In [ ]:
df_final.shape

---

<a id='section7'></a>

## <a id='#section7'>7. Exploratory Data Analysis</a>

In [ ]:
df_final_2021 = df_final[df_final['Season'] == '2020/2021']

In [ ]:
df_final_2021.head(10)

Next, we will check to see how many missing values we have i.e. the number of NULL values in the dataset, and in what features these missing values are located. This can be plotted nicely using the [missingno](https://pypi.org/project/missingno/) library (pip install missingno).

In [ ]:
# Plot visualisation of the missing values for each feature of the DataFrame
msno.matrix(df_final_2021, figsize = (30, 7))

In [ ]:
# Counts of missing values
null_value_stats = df_final_2021.isnull().sum(axis=0)
null_value_stats[null_value_stats != 0]

The visualisation shows us very quickly where the null values are. The first block of white is empty for goalkeeper statistics, which is the majority of the players.

We can also see that currently, the second block represents missing values for Capology data.

---

<a id='section5'></a>

## <a id='#section8'>8. Exporting Datsets</a>
Export the two datasets - Golden ID and the Final Player Valuation Dataset

In [ ]:
# Export DataFrame as a CSV file

## Golden ID
df_golden_id_select.to_csv(data_dir + f'/export/' + f'golden_id.csv', index=None, header=True)

## Final Player Performance Valuation
df_final_2021.to_csv(data_dir + f'/export/' + f'player_performance_valuation_latest.csv', index=None, header=True)

## <a id='#section9'>9. Summary</a>
This notebook joins the scraped and engineered player datasets inclduing the performance dataset from [FBref](https://fbref.com/en/) (provided by [StatsBomb](https://statsbomb.com/)), [TransferMarkt](https://www.transfermarkt.co.uk/) estimated player values and [recorded transfer](https://github.com/ewenme/transfers) datasets, and player salaries dataset from [Capology](https://www.capology.com/), through the [record-linkage](https://pypi.org/project/recordlinkage/) library, to create one, unified source of information, that can be used for for further analysis of players performance statistics and financial valuations, to be used as part of an Expected Transfer (xTransfer) model.

With this notebook we now have aggregated player performance data for players in the 'Big 5' European leagues for the 17/18-present seasons, along with their corresponding [TransferMarkt](https://www.transfermarkt.co.uk/) valuations and contractual information.

## <a id='#section10'>10. Next Steps</a>
These final datasets are now ready for any further analysis including modeling and data visualisation.

## <a id='#section11'>11. References</a>

#### Fuzzy Matching through Record Linkage
*    Record Linkage Wiki: https://en.wikipedia.org/wiki/Record_linkage
*    [Official Record Linkage documentation web](https://recordlinkage.readthedocs.io/en/latest/about.html)
*    [Official Record Linkage documentation PDF](https://readthedocs.org/projects/recordlinkage/downloads/pdf/stable/)
     -    [0. Preprocessing](https://recordlinkage.readthedocs.io/en/latest/ref-preprocessing.html)
     -    [1. Indexing](https://recordlinkage.readthedocs.io/en/latest/ref-index.html)
     -    [2. Comparing](https://recordlinkage.readthedocs.io/en/latest/ref-compare.html)
     -    [3. Classification](https://recordlinkage.readthedocs.io/en/latest/ref-classifiers.html)
     -    [4. Evaluation](https://recordlinkage.readthedocs.io/en/latest/ref-evaluation.html)
*    Evaluation:https://recordlinkage.readthedocs.io/en/latest/ref-evaluation.html
*    Python Tools for Record Linking and Fuzzy Matching by Chris Moffitt: https://pbpython.com/record-linking.html
*    [Python Tools for Record Linking and Fuzzy Matching](https://pbpython.com/record-linking.html). For more information for record linking and fuzzy matching.
*    [Python Package Index (PyPI) Jellyfish library](https://pypi.org/project/jellyfish/).

---

***Visit my website [eddwebster.com](https://www.eddwebster.com) or my [GitHub Repository](https://github.com/eddwebster) for more projects. If you'd like to get in contact, my Twitter handle is [@eddwebster](http://www.twitter.com/eddwebster) and my email is: edd.j.webster@gmail.com.***

[Back to the top](#top)